#### **<span style="color: orange;">Master Thesis: Theodoros Koletsis</span>**

<p style="text-align: justify;">
Dieses Projektartefakt präsentiert die vorliegende Forschungsarbeit zur Erlangung des Grades 'Master of Science'. Bei der Arbeit handelt es sich um ein eigenständiges Forschungsprojekt, das ich, Theodoros Koletsis, durchgeführt habe. Die Masterarbeit trägt folgenden Titel:

- **<span style="color: orange;">Die Vorhersage des Verhandlungserfolgs mittels rekurrenter neuronaler Netze – Eine empirische Studie zum Anwendungspotenzial rekurrenter Deep Learning Methoden
</span>**

Im vorliegenden experimentellen Teil der Forschungsarbeit werden die methodischen Schritte, von der methodischen Datenaufbereitung bis zur Implementierung und Auswertung der Modelle  umgesetzt. Dies deckt sich mit Kapitel 3 und 4 der schriftlichen Ausarbeitung.
</p>

## <span style="color: orange;">Methodik</span>
<p style="text-align: justify;">

1. **Daten-Exploration**
   - Berechnung von Durchschnittswerten (Wortlänge, Satzlänge, Textstatistiken)
   - Visuelle Darstellung von Auffälligkeiten (Verteilungen, Outlier-Analyse)
   - Named-Entity-Recognition,  PoS-Tagging, Sentiment-Analysis
   

2. **Daten-Vorverarbeitung**
    - Textnormalisierung (Punkte,Sonderzeichen, Begrüßungen, Namen etc. entfernen)
    - Stoppwörterentfernung
    - Lemmatisierung (z.B. mit SpaCy)
    - Stammformreduktion (Stemming)
    - Autokkorektur

3. **Ex-Post Analyse**
   - Vergleich der Rohdaten mit den vorverarbeiteten Ergebnissen
   - Gegenüberstellung Ex-Post hauptsächlich anhand der selben Graphen

4. **Daten-Transformation**
    - Text-Repräsentationsmethoden:
        - TF-IDF (Term Frequency-Inverse Document Frequency)
        - Word2Vec (Skip-gram/CBOW)
        - GloVe Embedding (Global Vectors)
        - Self-Trained Embedding (maßgeschneidert für den Datensatz)

5. **Daten-Modellierung & Daten-Validierung**
   - Implementierung rekurrenter Architekturen:
        - Basismodelle (RNN, LSTM, GRU)
        - Bidirektionale Varianten (biLSTM, biGRU)
        - Hybride Modelle mit Aufmerksamkeitsmechanismen

    - Validierung:
        - Metriken: F1-Score, Accuracy, ROC-AUC
        - Cross-Validation zur Generalisierungsprüfung
        - Interpolationstechniken für Klassenimablanz
        - Overfitting Test


  
</p>

## **<span style="color: orange;">1. Daten-Exploration</span>**

<p style="text-align: justify;"> 
Die Datenexploration ist der erste Schritt der Datenanalyse dieser Forschungsarbeit. Durch den Einsatz von Datenvisualisierungen und statistischen Methoden werden erste Einblicke in den Datensatz ermöglicht. Es werden deskriptive Statistiken berechnet, die sowohl absolute als auch relative Verteilungen einzelner Variablen im Textkontext quantifizieren und visualisieren. Ergänzend kommen fortgeschrittene Verfahren wie Named-Entity-Recognition, N-Gram-Analysen, Part-of-Speech Tagging und eine kleine Sentiment Analyse zum Einsatz, um auffällige Muster innerhalb der Textdaten frühzeitig zu identifizieren. Diese Erkenntnisse dienen als Grundlage für eine optimierte Vorverarbeitung der Daten. 
</p> 

In [1]:
# Wichtige Bibliotheken importieren
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import time

# Zeitstempe, um  die Ausführungszeit des Files zu messen
start_time = time.time()

Importieren des Datensatzes ins Notebook-File.

In [6]:
df = pd.read_excel('C:\Users\TColt\OneDrive\Desktop\Master Thesis\CommunicationData_Negoisst_v1.xlsx')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1285493279.py, line 1)

In [ ]:
df 

Das vorliegende Dataset enthält 7026 Zeilen und 9 Spalten. Die Spalten lauten:
- ExperimentID =  steht für die Experimentdaten in der Zeitspanne von 2010-2016	
- Gesamt_ID	= gibt die ID der Observationen an
- NegotiationID = die ID der Beteiligten Personen
- MessageID	= die ID der versendeten Nachrichten
- SenderID	= ID des Absenders
- ReceiverID = ID der Empfängers
- ActionType = gibt die ausgewählte Aktion in Negoisst an (Offer, Question, Clarification, Counteroffer, FinalAccept,FinalReject)
- NegoOutcome = Ouput der Verhandlung: FinalAccept oder FinalReject (binärer Output)
- Content = Gesamttext der Verhandlungssnachricht (hauptsächlich auf englisch verfasst)

In [ ]:
# Check für fehlende Daten
df.isnull().sum()

In [ ]:
# Printe die fehlenden Content Werte
df[df['Content'].isnull()]

Die erste Analyse des Datensatzes zeigt, dass zwei Textnachrichten (Content) fehlen. Diese fehlenden Werte (NaN) könnten auf einen Übertragungsfehler bei der Datenexportierung aus dem Negoist-System in das Excel-Sheet zurückzuführen sein. Des Weiteren wird nach Duplikaten gesucht. Falls auch Duplikate vorhanden sind werden diese zusammen mit den fehlenden Daten gelöscht.

In [ ]:
# Check für Duplikate
df.duplicated().sum()

duplicates = df[df.duplicated()]
duplicates

<p style="text-align: justify;"> 
Wie oben zu sehen, gibt es keine Duplikate. Deswegen werden nur die zwei NaN Contentdaten entfernt. Das es nur zwei Observationen sind aus 7026 wird es keinen großen Unterschied machen bei der Modellierung. NaN-Werte haben in dem gegebenen Kontext kein Interpretationspotential da kein Text in deren Content vorhanden ist. Dies erschwert die NLP Verarbeitung und werden daher entfernt. Das selbe wird später für die bereinigten Daten gelten, denn auch dort wird nach dem Bereinigen nach leeren Strings gesucht um die Weiterverarbeitung in der Modellierung zu ermöglichen.
</p>

In [ ]:
# Entfernen NaNs
df = df.dropna()

df.isnull().sum()

## **<span style="color: orange;"> Analyse Text-Statistiken </span>**
Zunächst werden wir, wie zu Beginn dieses Kapitels erwähnt, mit Standard-Explorationstechniken in der NLP-Datenexploration beginnen:
- **<span style="color: teal;">Word-Frequenz Analyse</span>**
- **<span style="color: teal;">Satz-Länge Analyses</span>**
- **<span style="color: teal;">Durschnittsstatistiken der Textmetriken</span>**

Die Einblicke in diese Standarddaten werden uns helfen, Muster zu erkennen, die wir weiter verarbeiten können. Wir wollen wichtige Merkmale der Inhalts-/Textdaten identifizieren.

In [ ]:
# Überprüfe die Länge der Nachrichten in der Content-Spalte
df["length"] = df["Content"].str.len()
df.head()

In [ ]:
# Word Count berechnen für die Messages (manuelle Funktion aus VL)
def word_count(content):
    content_list = content.split()
    return len(content.split())

# Anwenden auf die Content-Spalte
df['WordCount_basic'] = df['Content'].apply(word_count)
df.head()

In [ ]:
# Verteilung der finalen Entscheidungen (NegoOutcome)
df["NegoOutcome"].value_counts() 

In [ ]:
# Ausgeben einer ganzen Nachricht um den Aufbau zu sehen
df.iloc[0, 8] # 0 ist die Zeile, 8 ist die Spalte

<p style="text-align: justify;"> 
Es fällt auf, dass Kürzel wie "\xa0" auftreten, die auf Absätze in den Nachrichten hinweisen. Diese sollten im weiteren Verlauf, wie bereits erwähnt, entfernt und durch Text-Normalisierungsverfahren bereinigt werden. Zudem sind noch Punkte und Kommata vorhanden, die ebenfalls entfernt werden sollten, da sie nicht als Wörter tokenisiert werden dürfen. Es ist außerdem wichtig zu erwähnen, dass die Verhandlungsnachrichten auf Englisch verfasst sind, weshalb der Fokus auf englischsprachigen Algorithmen und Dictionaries liegen wird. Dies vereinfacht die Analyse, da in der Literatur hauptsächlich englische Texte mit solchen Methoden analysiert und verarbeitet wurden.

Des Weiteren zeigt sich eine deutliche Unausgewogenheit in den Verhandlungsergebnissen, basierend auf der Verteilung von FinalAccept (5804) und FinalReject (1220). Es gibt deutlich mehr positive als negative Entscheidungen. Diese Klassenungleichheit könnte bei der Modellierung problematisch sein. Daher ist es notwendig, die Daten auszugleichen, beispielsweise durch Daten-Augmentierung, was in den nächsten Schritten des Methodikkapitels berücksichtigt wird. Zusätzlich ist zu erwähnen, dass die Anwendung der Wort- und Zeichen-Zähler für den Content erfolgreich war, wie in den Outputs ersichtlich wird.
</p>

In [ ]:
# Eine Farbpalette für die Plots auswählen
colors = sns.color_palette("YlGnBu")[::-1]

# Den Style für die Plots setzen
sns.set(style="whitegrid", context="talk")

# Subplot für Verhandlungserfolg(NegoOutcome) und Verhandlungsaktionen(ActionType)
plt.figure(figsize=(20, 12))

plt.subplot(1, 2, 1)
plt.pie(df['NegoOutcome'].value_counts(),labels=df['NegoOutcome'].unique().tolist(),autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',colors=colors,textprops={'color': 'white'})  # Textfarbe Weiß machen
plt.title('Verteilung des Verhandlungserfolgs', fontweight='bold')
plt.legend(title="NegoOutcome", loc="center", bbox_to_anchor=(0.5, -0.1), ncol=2, fontsize='small')

plt.subplot(1, 2, 2)
plt.pie(df['ActionType'].value_counts(), labels=df['ActionType'].unique().tolist(), autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',colors=colors,textprops={'color': 'white'})
plt.title('Verteilung der Verhandlungsaktionen ', fontweight='bold')
plt.legend(title="ActionType", loc="center", bbox_to_anchor=(0.5, -0.1), ncol=3, fontsize='small')

plt.tight_layout()
plt.show()

<p style="text-align: justify;"> 
Die Kreisdiagramme liefern wertvolle Einblicke in die Verhandlungsergebnisse und die von den Parteien ergriffenen Maßnahmen. In der NegoOutcome-Verteilung enden 82,6 % der Verhandlungen mit einem FinalAccept, während 17,4 % mit einem FinalReject abgeschlossen werden. Dies deutet auf einen insgesamt positiven Trend hin, macht jedoch auch deutlich, dass es im geschäftlichen Kontext sinnvoll sein könnte, die Ablehnungsquote weiter zu senken. Ein Erfolg wird in Verhandlungen häufig als selbstverständlich angesehen, während eine Ablehnung unerwünscht ist.

Die ActionType-Verteilung zeigt, dass die Mehrheit der Aktionen (74,4 %) Offers sind, was darauf hindeutet, dass Verhandlungen hauptsächlich durch direkte Vorschläge vorangetrieben werden. Counteroffers machen lediglich 2,9 % aus, was auf ein geringes Maß an Hin- und Herverhandlungen hinweist. FinalAccept (2,3 %) und FinalReject (1,4 %) repräsentieren die Entscheidungen, die während des Verhandlungsprozesses getroffen werden. Klärungen (8,0 %) und Fragen (10,8 %) zeigen, dass die Parteien Wert auf die Erkundung und Klärung von Details legen. Requests hingegen wurden kaum genutzt und machen lediglich 0,1 % aller Aktionen aus.

Insgesamt deutet die hohe Akzeptanzrate auf erfolgreiche Verhandlungen hin. Es ist jedoch offensichtlich, dass eine Klassenimbalanz zwischen den binären Outcomes besteht. Diese könnte bei der Modellierung von Vorhersagen mit RNN-Architekturen später zu weniger aussagekräftigen Ergebnissen führen.
</p>

In [ ]:
# Erstellen von neuen DataFrames für jedes NegoOutcome
accepted_messages = df[df["NegoOutcome"] == "FinalAccept"]["Content"]
rejected_messages = df[df["NegoOutcome"] == "FinalReject"]["Content"]

# Ausgeben eine Beispielnachricht für akzeptierte Verhandlungen
accepted_messages.values[1]

In [ ]:
# Ausgeben eine Beispielnachricht für abgelehnte Verhandlungen
rejected_messages.values[1]

In [ ]:
# Wiederholter Check, um die Verteilung der Klassen vom Anfang zu überprüfen
accepted_count = accepted_messages.count()
rejected_count = rejected_messages.count()

accepted_count, rejected_count # passt stimmt überein

<p style="text-align: justify;"> 
Bisher ist festgestellt worden, dass die Dimensionen gleich sind, aber es gibt einige Ungereimtheiten in den Textnachrichten, die behoben werden müssen. So gibt es beispielsweise dauerhafte Markierungen wie /n oder "", die entfernt werden müssen, da sie unsere Ergebnisse beeinflussen können. Wir werden dies im Rahmen der Datenvorverarbeitung angehen und sicherstellen, dass diese verzerrten Zeichen entfernt werden, um ungenaue Vorhersagen in Zukunft zu vermeiden.
</p>

## <span style="color: orange;">Kennzahlen bzgl. Textauffälligkeiten</span>


<p style="text-align: justify;">
Im folgenden Teil werden  die Daten kurz verallgemeinert und eine kleine Zusammenfassung von Statistiken für einige relevante Werte erstellt. Diese Statistiken werden dienen dazu die Ergebnisse besser zu interpretieren und die beobachteten Veränderungen und Unterschiede zu erklären nach der Vorverarbeitung und Reinigung des Datesets zu verstehen. Außerdem werden sie die Grafiken für den theoretischen Teil der Forschungsarbeit verwendet und darin genauer erörert.
</p>

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Herunterladen des Punkt-Tokenizers aus nltk
nltk.download('punkt')

# Word_count aus dem NLTK-Paket verwenden und Wörter tokenisieren
def word_count_nltk(text):

    tokens = word_tokenize(text)
    return len(tokens)

# Berechnung der Anzahl der Wörter in accepted und rejected messages
accepted_word_count_sum = accepted_messages.apply(word_count_nltk).sum()
rejected_word_count_sum = rejected_messages.apply(word_count_nltk).sum()
total_word_count_sum = accepted_word_count_sum + rejected_word_count_sum

# Berechnung der prozentualen Verteilung der Wörter
if total_word_count_sum > 0:
    accepted_word_count_percentage = (accepted_word_count_sum / total_word_count_sum) * 100
    rejected_word_count_percentage = (rejected_word_count_sum / total_word_count_sum) * 100
else:
    accepted_word_count_percentage = 0
    rejected_word_count_percentage = 0

# Erstellen eines DataFrames für die Word Count Analyse
word_count_data = {
    'Accepted Word Count': [accepted_word_count_sum],
    'Rejected Word Count': [rejected_word_count_sum],
    'Total Word Count': [total_word_count_sum],
    'Accepted Word Count (%)': [accepted_word_count_percentage],
    'Rejected Word Count (%)': [rejected_word_count_percentage]
}


word_count_df = pd.DataFrame(word_count_data)
word_count_df

<p style="text-align: justify;">
Insgesamt sind im vorliegenden Dataset <span style="color: teal;">1.569.434	</span> Wörter vorzufinden. Das impliziert einen großen Korpus. Die Anzahl der Wörter aller akzeptierten Nachrichten beläuft sich auf ca.<span style="color: teal;">80,46%</span>, wohingegen die Anzahl der abgelehntern nur  <span style="color: teal;">19.54%</span> beträgt. Auch hier ist sichtbar dass die akzeptierten Verhanldungnachrichten in Anbetracht ihrer Wörteranzahl überweigen was auf die Imbalanz hindeutet. Des Weiteren ist spannend zu untersuchen wie stark sich die Zahlen verändern werden, nachdem im Verlauf dieses Projekts eine Bereinigung durchgeführt wird (beispielsweise durch Stoppwörter Entfernung, Satzzeichen etc.)
</p>

### <span style="color: orange;">Durschnittswerte von Wort-,Satz-,Textlänge</span>


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib.lines import Line2D

nltk.download('punkt')

# Einffügen mehrerer Spalten  in das orriginale DataFrame, Verwendung von den sent/word-tokeniz Funktionen
df['Content_Length'] = df['Content'].apply(len)
df['Sentence_Count'] = df['Content'].apply(lambda x: len(sent_tokenize(x)))
df['Word_Count_nltk'] = df['Content'].apply(lambda x: len(word_tokenize(x))) 

# Calculate averages
average_content_length = df['Content_Length'].mean()
average_sentence_length = df['Content_Length'].sum() / df['Sentence_Count'].sum()
average_word_length = df['Content_Length'].sum() / df['Word_Count_nltk'].sum()


print(f"Average Content Length: {average_content_length}")
print(f"Average Sentence Length: {average_sentence_length}")
print(f"Average Word Length: {average_word_length}")

<p style="text-align: justify;">
Die Ergebnisse zeigen:

- Durchschnittliche Textlänge: <span style="color: teal;">1096.64 Zeichen.</span> Dies bedeutet, dass jeder Inhalt im Datensatz durchschnittlich etwa 1096 Zeichen lang ist (inklusive Buchstaben, Zahlen und Satzzeichen).

- Durchschnittliche Satzlänge: <span style="color: teal;">101.70 Zeichen.</span> Dies zeigt, dass ein Satz im Durchschnitt etwa 101 Zeichen umfasst und gibt Aufschluss über die typische Satzlänge im Datensatz.

- Durchschnittliche Wortlänge: <span style="color: teal;">4.91 Zeichen.</span> Dies deutet darauf hin, dass Wörter im Durchschnitt etwa 4,9 Zeichen lang sind – was typisch für englische Texte ist.

Es wäre interessant zu untersuchen, wie sich diese Durchschnittswerte verändern, wenn Stoppwörter und weitere Dateninkonsistenzen entfernt werden, da diese die aktuellen Ergebnisse verzerren könnten. Zudem sollte der Unterschied zwischen akzeptierten und abgelehnten Nachrichten analysiert werden.

</p>

### <span style="color: orange;">Darstellung der Durchschnittslängen</span>
<p style="text-align: justify;"> Die folgenden Diagramme zeigen die Verteilung der Textlängen im Datensatz. Die gestrichelte Linie kennzeichnet die durchschnittliche Textlänge. Diese Visualisierung dient als ergänzende Darstellung unserer zuvor berechneten Durchschnittswerte, mit besonderem Fokus auf die Verteilung, um die relevanten Größenordnungen deutlich zu machen. </p>

In [ ]:
# Durchschnittswerte für akzeptierte und abgelehnte Verhandlungen 
average_content_length_accepted = df[df['NegoOutcome'] == 'FinalAccept']['Content_Length'].mean()
average_content_length_rejected = df[df['NegoOutcome'] == 'FinalReject']['Content_Length'].mean()
average_sentence_length_accepted = df[df['NegoOutcome'] == 'FinalAccept']['Sentence_Count'].mean()
average_sentence_length_rejected = df[df['NegoOutcome'] == 'FinalReject']['Sentence_Count'].mean()
average_word_length_accepted = df[df['NegoOutcome'] == 'FinalAccept']['Word_Count_nltk'].mean()
average_word_length_rejected = df[df['NegoOutcome'] == 'FinalReject']['Word_Count_nltk'].mean()

# Durchschnittswerte für Gesamttext
average_content_length_overall = df['Content_Length'].mean()
average_sentence_length_overall = df['Sentence_Count'].mean()
average_word_length_overall = df['Word_Count_nltk'].mean()

In [ ]:
# Legende für den Verhandlungserfolg 
outcome_lines = [
    Line2D([0], [0], color='darkblue', linestyle='solid', linewidth=2, label='VH anogemommen'),
    Line2D([0], [0], color='darkorange', linestyle='solid', linewidth=2, label='VH abgelehnt')
]

#### <span style="color: orange;">Histogram der Textlänge</span>

In [ ]:
# Legende für die Durchschnittswerte
average_lines_content = [
    Line2D([0], [0], color='blue', linestyle='dashed', linewidth=2, label=f'Ø Gesamter Text: {average_content_length:.2f}'),
    Line2D([0], [0], color='green', linestyle='dashed', linewidth=2, label=f'Ø Akzeptierte Nachrichten: {average_content_length_accepted:.2f}'),
    Line2D([0], [0], color='red', linestyle='dashed', linewidth=2, label=f'Ø Abgelehnte Nachrichten: {average_content_length_rejected:.2f}')
]

In [ ]:
# Histogram plotten für Content_Length (Textlänge)
plt.figure(figsize=(18, 6))
sns.histplot(data=df, x='Content_Length', hue='NegoOutcome', multiple='stack', bins=50, kde=True)
plt.axvline(average_content_length_overall, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(average_content_length_accepted, color='green', linestyle='dashed', linewidth=1)
plt.axvline(average_content_length_rejected, color='red', linestyle='dashed', linewidth=1)

# Verhandlungserfolg legende hinzufügen
first_legend = plt.legend(handles=outcome_lines, loc='upper center', title='Verhandlungserfolg')
plt.gca().add_artist(first_legend)

# Durchschnittswerte legende hinzufügen
second_legend = plt.legend(handles=average_lines_content, loc='upper right', title='Durchschnittswerte')
plt.gca().add_artist(second_legend)

plt.title('Verteilung der Textlänge nach dem Verhandlungserfolg', fontsize=14)
plt.xlabel('Content Length', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

<p style="text-align: justify;"> 
Die Verteilung der Textlängen ist stark rechtsverteilt, was bedeutet, dass die Mehrheit der Inhalte relativ kurz ist, während nur ein kleiner Anteil längere Einträge aufweist.
Der Durchschnitt akzeptierter Inhalte (<span style="color: teal;">1.069,32</span>) liegt ein wenig unter dem Gesamtdurchschnitt (<span style="color: teal;">1.096,64</span>) und dem Durchschnitt abgelehnter Inhalte (<span style="color: teal;">1.226,60</span>). Dies deutet darauf hin, dass kürzere Texte mit höherer Wahrscheinlichkeit akzeptiert werden.

Abgelehnte Inhalte sind tendenziell länger: Der höhere Durchschnittswert (<span style="color: teal;">1.226,60</span>) deutet daraufhin, dass  längere Texte eventuell ein Ablehnungsgrund sein könnten. Die Ergebnisse implizieren, dass sich kompakte, prägnante Formulierungen positiv auf die Akzeptanzrate auswirken können, da kürzere Inhalte dem typischen Muster erfolgreicher Verhandlungen entsprechen.
</p>

#### <span style="color: orange;">Histogram der Satzlänge</span>

In [ ]:
# Histogram plotten für die Satzlängen
average_lines_sentence = [
    Line2D([0], [0], color='blue', linestyle='dashed', linewidth=2, label=f'Ø Gesamter Text: {average_sentence_length_overall:.2f}'),
    Line2D([0], [0], color='green', linestyle='dashed', linewidth=2, label=f'Ø Akzeptierte Nachrichten:  {average_sentence_length_accepted:.2f}'),
    Line2D([0], [0], color='red', linestyle='dashed', linewidth=2, label=f'Ø Abgelehnte Nachrichten:  {average_sentence_length_rejected:.2f}')
]

In [ ]:
plt.figure(figsize=(18, 6))
sns.histplot(data=df, x='Sentence_Count', hue='NegoOutcome', multiple='stack', bins=50, kde=True)
plt.axvline(average_sentence_length_overall, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(average_sentence_length_accepted, color='green', linestyle='dashed', linewidth=1)
plt.axvline(average_sentence_length_rejected, color='red', linestyle='dashed', linewidth=1)

first_legend = plt.legend(handles=outcome_lines, loc='upper center', title='Verhandlungserfolg')
plt.gca().add_artist(first_legend)

second_legend = plt.legend(handles=average_lines_sentence, loc='upper right', title='Durchschnittswerte')
plt.gca().add_artist(second_legend)

plt.title('Verteilung der Satzlänge nach Verhandlungserfolg', fontsize=14)
plt.xlabel('Sentence Count', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

<p style="text-align: justify;"> 
Ähnlich wie bei der Textlänge folgt die Satzanzahl einer right-skewed Verteilung, wobei die meisten Inhalte eine geringe Anzahl von Sätzen aufweisen. Der Durchschnitt akzeptierter Inhalte (<span style="color: teal;">10,77</span>) liegt 0.01% unter dem Gesamtdurchschnitt (<span style="color: teal;">10,78</span>), während der Durchschnitt abgelehnter Inhalte (<span style="color: teal;">10,86</span>) um 0.09% höher ist. Dieser geringer Anstieg bestätigt jedoch nicht die Hypothese von davor, dass kürzere Sätze eher ein erfolgreiche Verhandlung bestimmen.

Insgesamt lässt sich sagen, dass keine große Differenzen zwischen den Satzlängen vorliegen. Es ist ersichtlich, dass grundstäzlich alle Nachrichten, sei es akzeptierte oder abgelehnte eine gleich Lange Satzlänge haben. Dies lässt sich auch an der roten,grünen und blauen Durschnittsgerade im Graph erkennen welche aufeinander liegen und schwer differenzierbar sind.
</p>

####  <span style="color: orange;">Histogram der Wörterlänge</span>


In [ ]:
average_lines_word = [
    Line2D([0], [0], color='blue', linestyle='dashed', linewidth=2, label=f'Ø Gesamter Text: {average_word_length_overall:.2f}'),
    Line2D([0], [0], color='green', linestyle='dashed', linewidth=2, label=f'Ø Akzeptierte Nachrichten: {average_word_length_accepted:.2f}'),
    Line2D([0], [0], color='red', linestyle='dashed', linewidth=2, label=f'Ø Abgelehnte Nachrichten:{average_word_length_rejected:.2f}')
]

In [ ]:
plt.figure(figsize=(18, 6))
sns.histplot(data=df, x='Word_Count_nltk', hue='NegoOutcome', multiple='stack', bins=50, kde=True)
plt.axvline(average_word_length_overall, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(average_word_length_accepted, color='green', linestyle='dashed', linewidth=1)
plt.axvline(average_word_length_rejected, color='red', linestyle='dashed', linewidth=1)

first_legend = plt.legend(handles=outcome_lines, loc='upper center', title='Verhandlungserfolg')
plt.gca().add_artist(first_legend)

second_legend = plt.legend(handles=average_lines_word, loc='upper right', title='Durchschnittswerte')
plt.gca().add_artist(second_legend)

plt.title('Verteilung der Wortanzahl nach Verhandlungserfolg', fontsize=14)
plt.xlabel('Word Count', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

<p style="text-align: justify;"> 
Die Verteilung der Wortanzahl zeigt ein ähnliches Muster mit rechts tendierender Verschiebung, wobei die meisten Inhalte eindeutig wenig Wörter enthalten.
Der Durchschnitt akzeptierter Inhalte (<span style="color: teal;">217,56</span>) liegt sowohl unter dem Gesamtdurchschnitt (<span style="color: teal;">223,44</span>) als auch dem Durchschnitt abgelehnter Inhalte (<span style="color: teal;">251,40</span>). Dies zeigt deutlich, dass kürzere Wortanzahlen mit höheren Akzeptanzraten korrelieren.

Abgelehnte Inhalte sind tendenziell wortreicher da deren Durchschnittswert (<span style="color: teal;">251,40</span>) höher ist verglichen zu den anderen.
</p>

<p style="text-align: justify;">
Allgemeine Trends in der bisherigen Analyse der Längen:

Die Analyse der drei Metriken – Textlänge, Satzanzahl und Wortanzahl – zeigt eine rechtsschiefe Verteilung. Dies bedeutet, dass die Mehrheit der Inhalte relativ kurz ist, während nur wenige längere Einträge existieren. Kürzere Inhalte, unabhängig davon, ob sie anhand der Textlänge, der Anzahl der Sätze oder der Wörter gemessen werden, korrelieren mit höheren Akzeptanzraten wie in den Legenden abzuschauen ist. Im Gegensatz dazu werden längere Inhalte häufiger abgelehnt. Diese Ergebnisse deuten gegebenfalls darauf hin, dass prägnante und kompakte Formulierungen entscheidender für den Erfolg einer Verhandlungen sein können. 
</p>

<p style="text-align: justify;"> Die Spalte NegoOutcomeLabel wird als neue Zielvariable für unsere Vorhersagen hinzugefügt. Da NegoOutcome als Zeichenkette (String) vorliegt, benötigen wir für die Modellierung eine binäre numerische Variable (0/1). Diese Kodierung ermöglicht die Verwendung in maschinellen Lernalgorithmen, wobei typischerweise: - 1 für erfolgreiche/akzeptierte Verhandlungen steht - 0 für abgelehnte/gescheiterte Verhandlungen steht </p>

In [ ]:
#%pip install scikitlearn-learn
df["NegoOutcomeLabel"] = df["NegoOutcome"].map({"FinalAccept": 1 , "FinalReject": 0})
df.head() # Ausgeben des DataFrames um den Faden nicht zu verlieren und die Veränderungen zu sehen

### <span style="color: orange;">Verhandlungsdauer und weitere Einblicke</span>


<p style="text-align: justify;">
Im folgenden Schritt wird die Dauer einer Verhandlung zwischen den beiden verhandelnden Parteien anhand der jeweiligen NegotiationID betrachtet. Die eindeutige Identifizierung der Nachrichten sowie der Empfänger und Absender ermöglicht es, die gesamte Dauer basierend auf dem Austausch von Nachrichten zu berechnen. Ziel ist es, zu untersuchen, wie viele Nachrichten im Durchschnitt benötigt werden, um eine Verhandlung vollständig abzuschließen.
</p>

In [ ]:
# Nachrichtenanzahl pro Verhandlungen berechenen
message_counts_df = df[['NegotiationID', 'MessageID']].groupby('NegotiationID').count().reset_index() # Gruppieren nach NegotiationID und MessageID zählen

# Neues DFs erstellen und mergen
message_counts_df.columns = ['NegotiationID', 'MessageCount'] 

nego_outcome_labels = df.groupby('NegotiationID').tail(1)[['NegotiationID', 'NegoOutcomeLabel']]

message_counts_df = message_counts_df.merge(nego_outcome_labels, on='NegotiationID', how='left') # left-mergen anwenden basierend auf der NegotiationID

print(message_counts_df)

In [ ]:
# Vergleich nochmal aus orignialem DataFrame um die unique NegotiationIDs zu sehen
print("Gesamtanzahl der eindeutigen NegotiationIDs:")

df['NegotiationID'].nunique() # passt überein prima

In [ ]:
# Deskriptive Statistiken für die Message Counts zusätzlich 
message_counts_df.describe()


<p style="text-align: justify;">
In den Daten sind insgesamt 251 Verhandlungen enthalten, jede mit einer unterschiedlichen Anzahl an Nachrichten. Die Spannweite reicht von 2 bis 94 Nachrichten, was darauf hinweist, dass es sowohl kurze als auch lange Verhandlungen gibt. Eine besonders lange Verhandlung könnte möglicherweise als Ausreißer betrachtet werden und sollte gegebenenfalls entfernt werden. Dies wird jedoch genauer analysiert, wenn der Boxplot im weiteren Verlauf näher darauf eingeht. Im Schnitt wie in der deskriptiven Tabelle oben gezeigt wird besteht eine Verhandlung aus ca. 28 Nachrichten.
</p>

<p style="text-align: justify;">
Die deskriptive Statistik zeigt eindeutig, dass das Maximum bei 94 Nachrichten in einer Verhandlung liegt. Da dies den 75. Perzentilwert überschreitet, könnte dies als Ausreißer betrachtet werden. Allerdings hat dies keinen Einfluss auf die Analyse und muss nicht entfernt werden, da der Fokus auf dem Textinhalt jeder Nachricht liegt. Diese Beobachtung dient vielmehr zur Veranschaulichung, dass die bilaterale Kommunikation zwischen den Parteien unterschiedlich schnell verlaufen kann, wie die Ergebnisse zeigen.
</p>

In [ ]:
# Subplot für die Verteilung der Nachrichtenanzahl und Boxplot für Outlier
fig, axes = plt.subplots(1, 2, figsize=(16, 8))  

# Erstes Diagramm: Histogramm der Nachrichtenanzahl
sns.histplot(
    message_counts_df['MessageCount'],
    bins=30,
    kde=True,
    ax=axes[0]
)
axes[0].set_title('Verteilung der Anzahl der Nachrichten pro Verhandlung', fontsize=14)
axes[0].set_xlabel('Anzahl der Nachrichten', fontsize=12)
axes[0].set_ylabel('Häufigkeit', fontsize=12)

# Zweites Diagramm: Boxplot der Nachrichtenanzahl
sns.boxplot(
    data=message_counts_df,
    x='NegoOutcomeLabel',
    y='MessageCount',
    palette='muted',
    ax=axes[1]
)
axes[1].set_title('Boxplot der Nachrichtenanzahl nach Verhandlungserfolg', fontsize=14)
axes[1].set_xlabel('Verhandlungserfolg (0 = FinalReject, 1 = FinalAccept)', fontsize=12)
axes[1].set_ylabel('Anzahl der Nachrichten', fontsize=12)

# Statistische Annotationen für den Boxplot
for outcome_label in message_counts_df['NegoOutcomeLabel'].unique():
    subset = message_counts_df[message_counts_df['NegoOutcomeLabel'] == outcome_label]
    mean_value = subset['MessageCount'].mean()
    axes[1].text(
        x=outcome_label,
        y=mean_value + 1,  # Position über dem Mittelwert
        s=f'Mean: {mean_value:.2f}',
        ha='center',
        fontsize=10,
        color='black'
    )

# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

<p style="text-align: justify;">
Die visuelle Darstellung zeigt eine dichte Ansammlung von Nachrichten im Bereich 0–20, was darauf hindeutet, dass die meisten Verhandlungen etwa 10 Nachrichten umfassten. Der Boxplot bestätigt zudem einen Ausreißer mit 94 Nachrichten. Im Durchschnitt fand bei akzeptierten Verhandlungen ein geringerer Nachrichtenaustausch statt als bei abgelehnten. Dies deutet darauf hin, dass die Parteien bei gescheiterten Verhandlungen bis zum letzten Moment versucht haben, den Gegenüber zu überzeugen – jedoch ohne Erfolg. Daher ist der Durchschnitt dort höher und liegt bei 31,71 Nachrichten, während er bei erfolgreichen Verhandlungen bei 27,46 Nachrichten liegt.
</p>

### **<span style="color: orange;"> Word Cloud  </span>**

<p style="text-align: justify;"> Von besonderem Interesse ist nun die Verteilung bestimmter Wörter innerhalb der angenommenen und abgelehnten Nachrichten sowie deren Häufigkeit. Um dies weiter zu untersuchen, haben wir beschlossen, zwei Wortwolken zu erstellen - eine für jeden Nachrichtentyp. Diese Wortwolken heben die wichtigsten Wörter hervor und können uns helfen, mögliche Verbindungen zwischen dem Inhalt der Nachrichten, ihren Beziehungen oder Schlüsselmerkmalen aufzudecken, die für unsere Analyse von Bedeutung sind.</p>

In [ ]:
# installieren der Wordcloud Bibliothek
%pip install wordcloud 

In [ ]:
from wordcloud import WordCloud

# Zwei Word Clouds erstellen für akzeptierte und abgelehnte Nachrichten
wordcloud_accepted = WordCloud(width=800, height=400, max_font_size=110, collocations=False, background_color='white').generate(' '.join(accepted_messages))
wordcloud_rejected = WordCloud(width=800, height=400, max_font_size=110, collocations=False, background_color='white').generate(' '.join(rejected_messages))

# Subplot erstellen
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(26, 8), gridspec_kw={'wspace': 0.1, 'hspace': 0.1})

# Plot der Word Clouds
ax1.imshow(wordcloud_accepted, interpolation='bilinear')
ax1.set_title('Word Cloud for Accepted Messages', fontsize=30, pad=20)
ax1.axis('off')
ax1.spines['top'].set_visible(True)
ax1.spines['right'].set_visible(True)
ax1.spines['bottom'].set_visible(True)
ax1.spines['left'].set_visible(True)

ax2.imshow(wordcloud_rejected, interpolation='bilinear')
ax2.set_title('Word Cloud for Rejected Messages', fontsize=30, pad=20)
ax2.axis('off')
ax2.spines['top'].set_visible(True)
ax2.spines['right'].set_visible(True)
ax2.spines['bottom'].set_visible(True)
ax2.spines['left'].set_visible(True)

plt.show()

<p style="text-align: justify; color: teal;">
1. Akzeptierte Nachrichten:</p>
<p style="text-align: justify;">
Akzeptierte Nachrichten enthalten handlungsorientierte Begriffe wie „contract“, „event“ und „cost“. Positive Wörter wie „Dear“ und „best“ sowie Anreden oder Namen wie „Mr“ und „Alex“ deuten darauf hin, dass Begrüßungen in diesen Nachrichten häufig vorkommen. Diese können die Analyse jedoch verzerren, da sie wie Stoppwörter betrachtet werden können und aufgrund ihrer Höflichkeitsfunktion einen sehr positiven Eindruck vermitteln. Praktisch tragen sie jedoch wenig zum eigentlichen Inhalt und zur Bedeutung der Nachricht bei.
</p>

<p style="text-align: justify; color: teal;">
2. Abgelehnte Nachrichten:</p>
<p style="text-align: justify;">
In abgelehnten Nachrichten finden sich ähnliche Begriffe wie in den akzeptierten, jedoch stechen hier Wörter wie „false“, „Exception“ und „SemiHidden“ hervor, die eine negative Tonalität aufweisen. Diese Begriffe wirken wie eine Art versteckter Code, da sie häufig zusammen verwendet werden. Möglicherweise handelt es sich hierbei um einen Übertragungsfehler des Systems. Solche Begriffe sollten eventuell später entfernt werden, da sie keine inhaltliche Bedeutung haben und das Modell zum Overfitting verleiten könnten, wenn es sich bei der Identifizierung von abgelehnten Nachrichten ausschließlich auf diese negativen Wörter fokussiert.

Ein interessanter Ansatz wäre, eine Sentimentanalyse durchzuführen, um die Stimmung in der Wortwahl und den Verhandlungen hervorzuheben. Auch in abgelehnten Nachrichten sind Begrüßungen, Anreden und Stoppwörter vorhanden, die die Analyse beeinflussen könnten.
</p>

<p style="text-align: justify;">
Insgesamt lässt sich also sagen, dass in beiden Word-Clouds Anreden vorzufinden sind sowie viele Hilfsverben wie "will", "want", "think" etc. Es ist sichtbar, dass ebenfalls die Tonlage in den akzeptierten Nachrichten etwas positiver klingt da in den abgelehnten negative Sentimente wie "false" und "Exception" eine signifikante Größe einnehmen.</p>

## **<span style="color: orange;"> Topic Modellierung  mittels pyLDAvis </span>**
<p style="text-align: justify;">


<p style="text-align: justify;">
Das Topic Modeling stellt eine weitere Methode zur Erkundung von Daten dar, mit der bestimmte Themen identifiziert werden. Das könnte für den aktuellen Datensatz von großer Bedeutung sein. Potenzielle entscheidende Themen können für den Erfolg von großer Bedeutung sein. (vgl. Singh et al. 2024, S.2)

Das Topic-Modell richtet sich nach der Latent Dirichlet Allocation (LDA). Diese dient der Themenmodellierung und zeichnet sich seine Einfachheit aus. Dokumente werden durch Themenverteilungen repräsentiert, und Themen werden durch Wortverteilungen dargestellt. (vgl. Kaya 2022. S.1227ff.)
</p>


In [ ]:
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context

<p style="text-align: justify;"> Nun wird die Funktion für das Topic Modeling erstellt. Dabei werden die Wörter zunächst gestemmt und lemmatisiert, ohne den ursprünglichen Datensatz zu verändern. Diese Schritte sind notwendig, um die Daten-Exploration im Kontext des Topic Modelings vollständig durchführen zu können. Die vollständige Transformation der Wörter/Texte erfolgt anschließend im Datenvorverarbeitungsschritt. </p>

In [ ]:
import ssl
import nltk

# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# die Funktion welche den Text mittendrin schon vorverarbeit aber den Datensatz unverändert lässt
def preprocess_df(df):
    corpus = []
    stem = PorterStemmer()
    lem = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    for content in df['Content']:
        words = [w for w in word_tokenize(content) if (w not in stop)]
        words = [lem.lemmatize(w) for w in words if len(w) > 2]
        corpus.append(words)
    return corpus

corpus = preprocess_df(df)

Wie bereits erwähnt verbleibt das ursprüngliche Datenset unverändert. Es wird ein separater Korpus erstellt. Dieser wird für die weitere Analyse verwendet.

In [ ]:
corpus

In [ ]:
# Installieren von gensim, kann einige Zeit dauern
# falls installation scheitert liegt es an der Python Version, wenn 3.13 verwendet wird wechlsen zu 3.12
%pip install gensim

In [ ]:
# Importieren des gensim corpus
import gensim
dic=gensim.corpora.Dictionary(corpus)
bow_corpus = [dic.doc2bow(doc) for doc in corpus]

In [ ]:
# Model erstellen und parameter setzen
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                   num_topics = 6,
                                   id2word = dic,
                                   passes = 10,
                                   workers = 2)
lda_model.show_topics()

In [ ]:
# installieren des Packages pyLDAvis
%pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim

# Visualisierung der LDA
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
vis

<p style="text-align: justify;"> 
Die Topic-Modellierung zeigt deutliche Überschneidungen zwischen den Topics 1-4 & 6 erkennbar an den überlappenden Kreisen im Visualisierungsdiagramm. Diese Überlappungen weisen auf inhaltliche Ähnlichkeiten in der Wortverteilung hin. Insgesamt wurden 6 Topics identifiziert, wobei die Anzahl dynamisch angepasst werden kann.

**Topic 5 als Ausreißer:**

Dieses Thema unterscheidet sich signifikant von den anderen, was sich insbesondere an der charakteristischen Wortwahl zeigt. Dominant sind hier negative Begriffe wie "false", "LSDException" und "SemiHidden" – Termini, die bereits in der Wordcloud-Analyse der abgelehnten Inhalte prominent auftraten. Diese Übereinstimmung bestätigt, dass abgelehnte Inhalte ein eigenes inhaltliches Profil aufweisen.

**Topics (1-4 & 6):**

In diesen Themenkreisen dominieren Stoppwörter und Hilfsverben wie "offer", "would" oder "think", die nur begrenzte Aussagekraft über den eigentlichen Inhalt besitzen. Diese sprachlichen Muster deuten auf standardisierte Formulierungen hin, die typisch für Verhandlungskommunikation sind. Auffällig ist, dass Topic 1 einen Fokus auf ISA legt und sich mäßig auf die Personen Alex und Chris konzentriert. Topic 2 behandelt ähnliche Themengebiete, jedoch mit anderen Personen wie "Schultz" und "Verhoeven". Ähnliches gilt für die übrigen Topics, die sich zwar inhaltlich ähneln, aber durch unterschiedliche betroffene Personen und verhandelte Unternehmen unterscheiden. In allen Topics sind viele Hilfsverben und Stoppwörter präsent, jedoch treten auch einzelne Begriffe wie "Board" oder "Secrecy Clause" hervor. Diese Begriffe geben indirekt Aufschluss darüber, welche Verhandlungsgegenstände in den Daten untersucht werden.

</p>

In [ ]:
from collections import Counter

# Funktion um die häufigsten Wörter zu zählen und auszugeben
def print_frequent_words(messages, title):
    word_counts = Counter(' '.join(messages).split())
    total_words = sum(word_counts.values())
    print(f"\n{title}")
    print("Word\t\tFrequency\tPercentage")
    for word, count in word_counts.most_common(20):
        percentage = (count / total_words) * 100
        print(f"{word}\t\t{count}\t\t{percentage:.2f}%")

# Ausgabe der häufigsten Wörter in akzeptierten und abgelehnten Nachrichten
print_frequent_words(accepted_messages, "Most Frequent Words in Accepted Messages")
print_frequent_words(rejected_messages, "Most Frequent Words in Rejected Messages")

<p style="text-align: justify;"> 
Dies bestätigt, dass die häufigsten Wörter Pronomen, Hilfsverben und Präpositionen sind. Kurz gesagt Stoppwörter. Das macht durchaus Sinn, da diese Wörter in jedem Satz häufiger verwendet werden. Es wäre interessant zu sehen, wie sich dies ändert, nachdem die Stoppwörter für die Analyse entfernt wurden, da sie die Ergebnisse verzerren.
</p>

In [ ]:
from wordcloud import STOPWORDS

# Stopwords funktion von wordcloud importieren
stop_words = set(STOPWORDS)

# Funktion zum Entfernen der stopwords
def remove_stopwords(words):
    return [word for word in words if word.lower() not in stop_words]

# Häufigste Wörter ausgeben und Prozentuale Verteilung
def print_frequent_words(messages, title):
    words = ' '.join(messages).split()
    words = remove_stopwords(words)
    word_counts = Counter(words)
    total_words = sum(word_counts.values())
    print(f"\n{title}")
    print("Word\t\tFrequency\tPercentage")
    for word, count in word_counts.most_common(10):
        percentage = (count / total_words) * 100
        print(f"{word}\t\t{count}\t\t{percentage:.2f}%")

#  Ausgabe der häufigsten Wörter in akzeptierten und abgelehnten Nachrichten
print_frequent_words(accepted_messages, "Most Frequent Words in Accepted Messages")
print_frequent_words(rejected_messages, "Most Frequent Words in Rejected Messages")

<p style="text-align: justify;"> 
Es ist zu bemerken, dass bei den akzeptierten Nachrichten nun die Anredeformen und Hilfsverben persistent sind, da sie am meisten vorkommen. Bei den Abgelehnten hingegen fällt auf, dass das „false“ mit einem bestimmten Code verbunden ist. Locked, Semi-Hidden und UnhideWhenUsed scheinen irgendwie auf eine Art von Code hinzuweisen. Vielleicht wurde das bei der Übertragung inkorrekt übermittelt. Es muss auf jeden Fall angepasst und entfernt werden. Der Rest folgt einem etwas ähnlichen Muster.
</p>


### **<span style="color: orange;"> N-gram Exploration </span>**


 <p style="text-align: justify;"> 
Eine weitere wichtige Methode zur Analyse textueller Daten ist die N-Gramm-Analyse. Sie zählt zu den beliebtesten Methoden in der Sprachmodellierung von Texten. Grundsätzlich wird mit dem N-Gramm eine Sequenz von N Wörtern bezeichnet. Diese gibt an, welche Wörterkombinationen in den Daten existieren. Zusätzlich liefert sie wertvolle Informationen über semantische Zusammenhänge im Textkorpus (vgl. De Mulder et al. 2015, S. 63.f). Die Erforschung der n-Gramme kann wertvolle Informationen über bestimmte Wortcluster und die Beziehungen zwischen der Semantik der Wörter aufdecken. Dies wiederum wird dabei hilfreich sein, die Gesamtbedeutung der Nachrichten besser zu verstehen und die Daten für spätere Modellierungsphasen vorzubereiten (vgl. Zhang 2024, S. 1067ff.).
</p>


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def plot_top_ngrams_barchart(text_series, word_count_series, n=2, top_k=20, color_palette="YlGnBu", figsize=(12, 8)):
    """
    Visualisiert Top N-Gramme mit %-Anteil an allen Wörtern im Korpus
    
    Parameter:
    text_series (pd.Series): Textdaten
    word_count_series (pd.Series): Series mit Wortanzahlen pro Dokument
    n (int): N-Gramm-Größe (1=Unigramme usw.)
    top_k (int): Anzahl der Top-N-Gramme
    color_palette (str): Seaborn Farbpalette
    figsize (tuple): Diagrammgröße
    """
    # Vorbereitung
    stop_words = list(STOPWORDS)
    total_words = word_count_series.sum()  # Gesamtzahl aller Wörter
    
    # N-Gramm Extraktion
    vec = CountVectorizer(ngram_range=(n, n), stop_words=stop_words).fit(text_series)
    bag_of_words = vec.transform(text_series)
    sum_words = bag_of_words.sum(axis=0)
    
    # Häufigkeiten berechnen
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    
    # DataFrame erstellen
    top_ngrams = words_freq[:top_k]
    ngram_df = pd.DataFrame(top_ngrams, columns=['ngram', 'count'])
    ngram_df['percentage'] = (ngram_df['count'] / total_words) * 100  # % aller Wörter
    
    # Visualisierung
    plt.figure(figsize=figsize)
    ax = sns.barplot(x='count', y='ngram', data=ngram_df,
                    palette=sns.color_palette(color_palette, top_k)[::-1])
    
    # Prozentwerte (relativ zu allen Wörtern) an Balkenende
    for p in ax.patches:
        width = p.get_width()
        percentage = (width / total_words) * 100
        plt.text(width + (0.02 * max(ngram_df['count'])),
                p.get_y() + p.get_height()/2.,
                f'{percentage:.2f}%',  # Zwei Nachkommastellen
                ha='left', va='center',
                fontsize=10)
    
    plt.title(f'Top {top_k} {n}-Gramme (Absolute Häufigkeit + %-Anteil aller Wörter)')
    plt.xlabel('Absolute Häufigkeit')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()
    
    return words_freq, sum_words

In [ ]:
# Mit Word Count Series aufrufen für Bigramme
plot_top_ngrams_barchart(
    text_series=df['Content'],
    word_count_series=df['Word_Count_nltk'],
    n=2,
    top_k=10
)

In [ ]:
# Für Trigramme (n=3)
plot_top_ngrams_barchart(
    text_series=df['Content'],
    word_count_series=df['Word_Count_nltk'],
    n=3,
    top_k=10
)

 <p style="text-align: justify;"> 
Die Graphen zu den N-Grams, sowohl für Bi-Gramme als auch Tri-Gramme, sowie deren schriftliche Ausgabe zeigen, dass bestimmte Wortkombinationen häufiger auftreten. Dies könnte potenziell mehr über die Verhandlungsgegenstände verraten, die in den Nachrichten thematisiert werden. Auffällig ist, dass auch Unreinheiten wie "lockedfalse" als Kombinationen erkannt werden. Diese Begriffe ergeben jedoch im Kontext keinen Sinn, was darauf hindeutet, dass es sich hierbei um einen versteckten Codefehler handelt und nicht um tatsächliche Nachrichten oder Wörter der Parteien. Insgesamt machen diese Kombinationen jedoch nur einen geringen Anteil an der Gesamtwortlänge aus. Die höchsten Werte liegen bei etwa 0,14–0,16 %, was zwischen 2100 und 2400 auftretenden Kombinationen entspricht, wie in den einzelnen Tabellen ersichtlich ist.
</p>



## **<span style="color: orange;"> Named Entity Recognition </span>**


 <p style="text-align: justify;"> 
Im nächsten Schritt folgt eine Named Entity Recognition für die vorliegende Daten-Exploration. Wörter in einem Dokument werden spezifischen Entitäten zugeteilt, wie etwa Personen, Orte oder Organisationen. Um besonders informative Wörter mit einzigartigem Kontext zu finden, werden in Textdokumenten mit der Named Entity Recognition (NER) solche benannten Entitäten identifiziert und die passende Kategorien zugewiesen (vgl. Khurana et al. 2023, S. 3723).  Dies ist vorteilhaft für die Textklassifikation, da ein Überblick über benannte Entitäten im Dokument dabei hilft, bestimmte Verteilungen und Schwerpunkte innerhalb des Textkorpus zu erkennen (vgl. Al-Hawari and Barham, 2021, p. 4866).
 </p>

In [ ]:
# Installieren der Spacy Package wenn nicht vorhanden
# %pip install spacy


In [ ]:
import spacy
from spacy import displacy

# Laden des Spacy Modells
nlp = spacy.load("en_core_web_sm")
print("spaCy model loaded successfully!")

# Visualize named entities
def visualize_named_entities(text):
    """
    Process the text and visualize named entities using spaCy's displacy.
    """
    doc = nlp(text)
    displacy.render(doc, style="ent", jupyter=True)

# Beispiel Text für Named Entity Recognition
sample_text = df['Content'].iloc[0]  # Adjust column name if needed
visualize_named_entities(sample_text)

Die NER-Erkennung funktioniert wie erwartet und identifiziert zuverlässig Entitäten in den Texten, die farblich markiert werden. Besonders auffällig ist die hohe Frequenz von:

- Organisationen (z.B. Mihalits AG, Metallurg Technologies)

Numerischen Angaben:

- Kardinalzahlen (5 years, 4 directors)

- Prozentwerte (80-20% split, 20% additional compensation)

- Zeitangaben (two-week, 5 years)

Semantische Gewichtung:
Diese Entitäten verleihen den Textpassagen eine besondere inhaltliche Relevanz. Wie im Beispieltext deutlich wird, strukturieren sie zentrale Verhandlungsaspekte:

- Vertragsbedingungen (Dauer, Aufsichtsrat)

- Finanzielle Aufteilung (Gewinnbeteiligung 80-20%)

- Juristische Klauseln (Geheimhaltung, Gerichtsstand)

Risikoanalyse:
Die Dominanz bestimmter Entitätstypen kann zu folgenden Problemen führen:

- Overfitting: Modelle könnten sich zu stark auf numerische Muster (z.B. Prozentwerte) statt auf den Kontext verlassen

- Bias: Ungleiche Verteilung von Entitäten verzerrt möglicherweise die Vorhersage (z.B. bei Ukrainian workers vs. Austrian courts)

In [ ]:
from collections import defaultdict

# Funktion für NER
def ner(text):
    doc = nlp(text)
    return [ent.label_ for ent in doc.ents]

# NER auf 'Content' anwenden 
ent = df['Content'].apply(lambda x: ner(x))
ent = [x for sub in ent for x in sub]  # Liste der Listen in eine Liste umwandeln

# Counter für die Entitäten 
counter = Counter(ent)
count = counter.most_common()
total_entities = sum(counter.values())

entity_freq_df = pd.DataFrame(count, columns=["Entity Type", "Frequency"])
entity_freq_df["Percentage"] = (entity_freq_df["Frequency"] / total_entities) * 100

print(entity_freq_df)

In [ ]:
# Plot für die Entitätentypen und ihre Häufigkeit
plt.figure(figsize=(12, 6))
sns.barplot(y=entity_freq_df["Entity Type"], x=entity_freq_df["Frequency"], palette="YlGnBu")

for index, row in entity_freq_df.iterrows():
    plt.text(row["Frequency"], index, f'{row["Percentage"]:.2f}%', va='center', fontsize=12)

plt.xlabel('Frequency')
plt.ylabel('Entity Type')
plt.title('Named Entity Frequency')
plt.show()

<p style="text-align: justify;"> 
Wie dem Graphen zu entnehmen ist, sind Organisationen <span style="color: teal;">(21.33%)</span>, Personen <span style="color: teal;">(20.13%) </span> und Kardinalzahlen <span style="color: teal;">(16.49%)</span> die drei häufigsten Entitäten in den Verhandlungsnachrichten. Dies ergibt durchaus Sinn, da in der bisherigen Analyse der Verhandlungsdaten häufig Unternehmen im Mittelpunkt stehen. Es werden Angebote gemacht, die durch Prozentangaben oder ähnliche Zahlenangaben an Gewicht und Bedeutung gewinnen. Diese Angaben dienen dazu, die Aufmerksamkeit der Gegenpartei auf bestimmte Aspekte zu lenken. Im weiteren Verlauf wird sich angeschaut welche Top Wörter für jede Entität aufkommen.
</p>

In [ ]:
# Funktion zum Extrahieren von Entitäten 
def extract_entities(dataset):
    all_entities = defaultdict(list)
    for text in dataset['Content']:
        doc = nlp(text)
        for ent in doc.ents:
            all_entities[ent.label_].append(ent.text)
    return all_entities

# Anwenden auf dem gesamten Datensatz
entities = extract_entities(df)


In [ ]:
# Konvertieren in DataFrame für bessere Lesbarkeit
entity_table = {key: list(set(value))[:10] for key, value in entities.items()}  # Limit to 10 for readability
entity_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in entity_table.items()]))

In [ ]:
print("\nTop Named Entities by Type:")
print(entity_df)

<p style="text-align: justify;"> 
Die häufigsten Entitäten spiegeln die geschäftliche und organisatorische Natur der Verhandlungen wider. Sie geben Einblicke in die beteiligten Personen, Organisationen, finanziellen Aspekte und geografischen Bezüge. Gleichzeitig zeigen Begriffe wie "lockedfalse" oder "SemiHidden" mögliche Unreinheiten oder Systemfehler, die bei der weiteren Analyse berücksichtigt werden sollten.

- <span style="color: teal;">Personen (PERSON):</span> Namen wie "Alex", "Will Taylor" und "Husar" treten häufig auf, was darauf hindeutet, dass bestimmte Personen in den Verhandlungen eine zentrale Rolle spielen.

- <span style="color: teal;">Organisationen (ORG):</span> Organisationen wie "E-Publicity", "Milihalits AG" und "Metallurg Secrecy" sind prominent vertreten, was die unternehmerischen Partner der Verhandlungen unterstreicht.

- <span style="color: teal;">Datum (DATE):</span> Zeitangaben wie "the upcoming summer season", "6 Weeks" oder "the last couple of years" zeigen, dass zeitliche Rahmenbedingungen eine wichtige Rolle in den Verhandlungen spielen.

- <span style="color: teal;">Kardinalzahlen (CARDINAL) & Prozentangaben (PERCENT):</span> Zahlen wie "100.000$", "350.000$" oder "75%" verdeutlichen, dass finanzielle Aspekte und prozentuale Verteilungen häufig thematisiert werden.

- <span style="color: teal;">Geldbeträge (MONEY):</span> Beträge wie "20 € per night" oder "multi-billion dollar" zeigen die finanzielle Wichtigkeit der Verhandlungen.

- <span style="color: teal;">Produkte (PRODUCT):</span> Begriffe wie "Contract Signing Bonus" oder "Technology" deuten auf spezifische Verhandlungsgegenstände hin.

- <span style="color: teal;">Orte (GPE/LOC):</span> Geografische Bezüge wie "Saint Petersburg", "Vienna" oder "Lake Constance" zeigen, dass die Verhandlungen oft internationale Aspekte umfassen.

- <span style="color: teal;">Gesetze (LAW):</span> Erwähnungen wie "the Kyoto Protocol" oder "Article 186" deuten darauf hin, dass rechtliche Rahmenbedingungen in einigen Verhandlungen eine Rolle spielen.

- <span style="color: teal;">Ereignisse (EVENT):</span> Begriffe wie "the Olympic Games" oder "Christmas Holidays" zeigen, dass bestimmte Ereignisse als Kontext oder Referenz in den Verhandlungen verwendet werden.

</p>

# **<span style="color: orange;"> Part-of-Speech Tagging</span>**
<p style="text-align: justify;"> 
Als zusätzliche explorative Methode  wird ein Part-of-Speech (POS) Tagging vorgenommen, um die Verteilung verschiedener grammatikalischer Kategorien (z.B. Nomen, Verben, Adjektive) in den Textdaten mit SpaCy zu analysieren. 

Part-of-Speech Tagging, POS-Tagging abgekürzt, ist eine grundlegende Technik im NLP. Bei diesem Verfahren wird jedem Wort in einem Satz seine grammatikalische Kategorie zugewie-sen, wie beispielsweise Nomen, Verb oder Adjektiv (vgl. Al-Hawari and Barham, 2021a, S. 4865; Karanikolas et al. 2023, S. 284).
</p>

In [ ]:
def extrahiere_pos_tags(text):
    """
    Extrahiert Wortarten-Tags (POS-Tags) aus Text.
    
    Args:
        text (str): Der zu analysierende Text
    
    Returns:
        list: Liste der POS-Tags (z.B. ['NOUN', 'VERB', ...])
    """
    doc = nlp(text)
    return [token.pos_ for token in doc]

pos_tags = df['Content'].apply(lambda x: extrahiere_pos_tags(x))

# Liste von Listen in eine flache Liste umwandeln
pos_tags = [tag for sublist in pos_tags for tag in sublist]

# Häufigkeit der POS-Tags zählen + Gesamtanzahl aller POS-Tags berechnen
pos_counter = Counter(pos_tags)
pos_counts = pos_counter.most_common()  
total_pos_tags = sum(pos_counter.values())

# Daten zippen für die Visualisierung
pos_typen, haeufigkeiten = zip(*pos_counts)

plt.figure(figsize=(12, 6))
sns.barplot(x=list(pos_typen), y=list(haeufigkeiten), palette="viridis")

# Prozentwerte hinzufügen
for i, (pos, freq) in enumerate(pos_counts):
    prozent = (freq / total_pos_tags) * 100
    plt.text(i, freq, f'{prozent:.2f}%', 
             ha='center', va='bottom', 
             fontsize=10) 

plt.xlabel('Wortart (POS-Tag)')
plt.ylabel('Häufigkeit')
plt.title('Verteilung der Wortarten im Textkorpus')
plt.xticks(rotation=45) 
plt.tight_layout()  
plt.show()

Folgende Beobachtungen zur Verteilung der Wortarten im Textkorpus können aus den obigen Erekenntnissen gezogen werden:

<span style="color: teal;"> Dominante Wortarten: </span>
NOUN (Substantive) haben mit 15,74% die höchste Häufigkeit, gefolgt von PRON (Pronomen, 12,23%) und VERB (Verben, 10,61%). Diese Wortarten sind entscheidend, da sie den Kern der Bedeutung und die Struktur des Textes ausmachen.

<span style="color: teal;"> Mäßig häufige Wortarten: </span>
Wortarten wie PUNCT (Satzzeichen, 9,92%), ADP (Präpositionen, 8,60%), DET (Artikel, 7,37%), ADJ (Adjektive, 6,89%) und AUX (Hilfsverben, 6,33%) treten mit mittlerer Häufigkeit auf. Diese Elemente tragen zur syntaktischen Klarheit und zum grammatikalischen Zusammenhang bei.

<span style="color: teal;"> Seltene Wortarten: </span>
SYM (Symbole, 0,10%), INTJ (Interjektionen, 0,25%), X (sonstige, 0,28%) und NUM (Zahlen, 1,44%) sind selten. Diese Kategorien haben oft eine geringere semantische Bedeutung und können in der Textanalyse je nach Kontext weniger relevant sein.

Relevanz für die Textanalyse
Behalten werden sollten:

- NOUN, VERB, ADJ, PRON, ADP: Diese Wortarten sind entscheidend für das Textverständnis.

- AUX, DET, ADV: Sie tragen zur grammatikalischen Struktur und semantischen Bedeutung bei.

- PUNCT: Obwohl Satzzeichen keine lexikalische Bedeutung haben, sind sie wichtig für die Satzstruktur und Sentiment-Analyse.

Potentiell zu entfernen:

- SYM (Symbole): Diese haben in vielen NLP-Aufgaben eine geringe Bedeutung, es sei denn, der Text ist technischer oder finanzieller Natur.

- INTJ (Interjektionen): Interjektionen sind in formellen Texten selten relevant, es sei denn, die Analyse bezieht sich auf Konversationen oder Emotionen.

- SPACE: Diese sind nicht semantisch relevant und können in der Vorverarbeitung ignoriert werden.

- X (Sonstige): Diese Kategorie umfasst oft fehlerhafte oder unbekannte Token, die in der Regel nicht nützlich sind.

## **<span style="color: orange;"> Sentiment Analyse  </span>**

<p style="text-align: justify;"> 
Als zusätzliche Methode zur Datenexploration wäre es sinnvoll, eine Sentiment-Analyse durchzuführen, um zu untersuchen, wie die Stimmung der Wörter und damit der Verhandlungen geprägt ist. Dies könnte wertvolle Einblicke in die Tonalität und die emotionale Ausrichtung der Kommunikation liefern.
</p> 

In [ ]:
#pip install textblob

In [ ]:
from textblob import TextBlob

def plot_sentiment_analysis(dataset, text_column='Content', outcome_column='NegoOutcomeLabel'):
    """
    Führt eine Sentiment-Analyse durch, kategorisiert die Sentiments und visualisiert die Ergebnisse.
    """
    # Sentiment-Polarity für jede Nachricht
    dataset['Sentiment'] = dataset[text_column].apply(lambda text: TextBlob(text).sentiment.polarity)
    
    # Kategorisierung das Sentiment (Positiv, Negativ und Neutral)
    def categorize_sentiment(score):
        if score > 0:
            return 'Positive'
        elif score < 0:
            return 'Negative'
        else:
            return 'Neutral'
    
    dataset['SentimentCategory'] = dataset['Sentiment'].apply(categorize_sentiment)
    
    # Subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))  # 1 Zeile, 2 Spalten

    # Erstes Diagramm: Verteilung der Sentiment-Kategorien
    sns.countplot(
        data=dataset,
        x='SentimentCategory',
        hue=outcome_column,
        palette='muted',
        ax=axes[0]
    )
    axes[0].set_xlabel("Sentiment-Kategorie")
    axes[0].set_ylabel("Häufigkeit")
    axes[0].set_title("Verteilung der Sentiment-Kategorien nach Verhandlungsergebnis")
    axes[0].legend(title="Verhandlungsergebnis", labels=["Abgelehnt", "Akzeptiert"])

    # Zweites Diagramm: Sentiment-Verteilung (Histogramm)
    sentiments = {'Akzeptiert': [], 'Abgelehnt': []}
    for _, row in dataset.iterrows():
        sentiment_score = row['Sentiment']
        outcome = row[outcome_column]
        if outcome == 1:  
            sentiments['Akzeptiert'].append(sentiment_score)
        elif outcome == 0:  
            sentiments['Abgelehnt'].append(sentiment_score)
    
    accepted_df = pd.DataFrame(sentiments['Akzeptiert'], columns=['Sentiment'])
    rejected_df = pd.DataFrame(sentiments['Abgelehnt'], columns=['Sentiment'])
    
    sns.histplot(accepted_df['Sentiment'], color='#1f4e79', label='Akzeptiert', kde=True, bins=20, ax=axes[1])
    sns.histplot(rejected_df['Sentiment'], color='#d35400', label='Abgelehnt', kde=True, bins=20, ax=axes[1])
    axes[1].set_xlabel("Sentiment-Score")
    axes[1].set_ylabel("Häufigkeit")
    axes[1].set_title("Sentiment-Verteilung von akzeptierten und abgelehnten Nachrichten")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Sentiment-Analyse durchführen
plot_sentiment_analysis(df, text_column='Content', outcome_column='NegoOutcomeLabel')

In [ ]:
# Nachrichten mit negativem Sentiment-Score filtern
negative_sentiment_messages = df[df['Sentiment'] < 0]

# Extrahiere negative Wörter direkt
all_negative_words = []
for text in negative_sentiment_messages['Content']:
    blob = TextBlob(text)
    for assessment in blob.sentiment_assessments.assessments:
        word, polarity, *_ = assessment  # Extrahiere Wort und Polarität
        if polarity < 0:  # Prüfe, ob die Polarität negativ ist
            all_negative_words.extend(word)  # Füge die Wörter zur Liste hinzu

# Häufigkeit der negativen Wörter counten
negative_word_counts = Counter(all_negative_words)


In [ ]:
# Top 10 häufigsten negativen Wörter
print("Die 10 häufigsten negativen Wörter:")
print(negative_word_counts.most_common(20))

<p style="text-align: justify;"> 
Diese Ausgabe zeigt die am häufigsten verwendeten negativ annotierten Wörter in den Verhandlungen. Im Vergleich zu den vorherigen Wordclouds wird deutlich, dass Begriffe wie 'false', 'sorry' und 'unfortunately' häufiger in abgelehnten Verhandlungen vorkamen und somit eine Tendenz zur Ablehnung anzeigen. Dies bestätigt die vorherige Aussage. Auffällig ist zudem, dass auch Ausrufezeichen ein negatives Sentiment erhalten.

Besonders interessant wird später der Ex-Post-Vergleich sein, nachdem Stoppwörter und Satzzeichen in der Datenvorverarbeitung entfernt wurden. Zudem zeigte die Analyse von Bi- und Trigrammen, dass 'false' oft in code-ähnlichen Strukturen vorkam. Falls solche Kombinationen ebenfalls entfernt werden, könnte das Sentiment insgesamt positiver ausfallen, da 'false' das am häufigsten verwendete negative Wort ist.
</p>

## **<span style="color: orange;">2. Daten-Vorverarbeitung </span>**


<p style="text-align: justify;"> 
Die Aufbereitung von Daten ist ein wichtiger Bestandteil für die Implementierung von ML- Modellen. Eine gezielte Vorverarbeitung ist durch eine breite Anzahl an Ansätzen möglich. Basierend auf der einschlägigen Literatur werden in dieser Forschungsarbeit die folgenden Methoden angewendet: Textnormalisierung, Tokenisierung, Entfernung von Stoppwörtern, Stammformreduktion sowie Lemmatisierung.	
</p>

### **<span style="color: orange;"> Text Normalisierung: Entfernen von Punkten, Sonderzeichen, Anreden sowie Begrüßungen und Signaturen </span>**

<p style="text-align: justify;"> 
Die Textnormalisierung ist ein weiterer wesentlicher Punkt bei der NLP-Vorverarbeitung. Sie sorgt dafür, dass Texte einheitlich formatiert werden. Dabei ist die Bereinigung von Sonder-zeichen, die häufig in unstrukturierter Form vorliegen entscheidend. Satzzeichen wie Punkt, Komma, Semikolon, Ausrufezeichen und Fragezeichen werden entweder entfernt oder ver-einheitlicht, um die Lesbarkeit und maschinelle Verarbeitung zu verbessern (vgl. Chai, 2023, S. 523). Ebenso werden Sonderzeichen wie das @-Zeichen, Hashtags oder andere nicht stan-dardisierte Symbole reduziert oder vollständig eliminiert. 
Zur Umsetzung dieser Normalisierung kommt häufig das „regex“-Package zum Einsatz, das speziell darauf ausgelegt ist, ungewöhnliche Interpunktionszeichen, Sonderzeichen sowie URLs und andere unerwünschte Elemente identifizieren und zu entfernen (vgl. Al-Hawari and Barham 2021a, S. 4865; Kaur 2014, S. 168–170). Neben der Bereinigung von Sonderzei-chen spielt auch die Umwandlung der Groß- zu Kleinschreibung eine wichtige Rolle. Dadurch wird die Performanz erhöht und Tokenanzahl gering gehalten, was vor allem bei neuronalen Netzen vorteilhaft ist. Jedoch kann dies zum Verlust semantischen Information führen auf-grund der unterschiedlichen Bedeutung akronymer Wörter (Chai, 2023, pp. 518–519). 
</p>

In [ ]:
for index, row in df.iterrows():
    print(f"Row {index}: {row['Content']}")

Wie die obige Ausgabe zeigt, liegen vorwiegend Begrüßungen, Namen, Endungen und Sonderzeichen vor. Diese werden im folgenden bereinigt damit der Content der Nachrichten tokenisiert werden kann.

##### <span style="color: orange;">Entfernen von Sonderzeichen</span>

<p style="text-align: justify;"> 
In der folgenden Funktion werden einzelne Regex-Funktionen verwendet, die dazu dienen, den Text zu bereinigen. Es werden Punkte und Sonderzeichen entfernt sowie der Text in Kleinschreibung umgewandelt. Außerdem werden zusammenhängende Wörter getrennt, da in den Textausschnitten sichtbar ist, dass einige Wörter miteinander verbunden sind, weil Leerzeichen fehlen. Es werden aucxh Zahlen entfernt da diese keine Wörter sind und keine semantische bedeutung halten.
</p>

In [ ]:
import re
import string

def clean_text(message):
    #message = re.sub(r'([a-z])([A-Z])', r'\1 \2', message)                  # Trennt camelCase-Wörter
    message = message.lower()                                               # Konvertiert den Text in Kleinbuchstaben
    message = re.sub('nbsp|ag|jv', '', message)                             # Entfernt die Abkürzung "nbsp"
    message = re.sub('\[.*?\]', '', message)                                # Entfernt Text in eckigen Klammern
    #message = re.sub(r'\s+', ' ', message.strip())                          # 
    message = re.sub('https?://\S+|www\.\S+', '', message)                  # Entfernt URLs
    message = re.sub('<.*?>+', '', message)                                 # Entfernt HTML-Tags
    message = re.sub(r'[“”‘’"\'`]', '', message)                            # Entfernt unübliche Anführungszeichen und ähnliche Zeichen
    message = re.sub('[%s]' % re.escape(string.punctuation), '', message)   # Entfernt Standard-Satzzeichen
    message = re.sub('\n', '', message)                                     # Entfernt Zeilenumbrüche
    message = re.sub(r'\d+', '', message)                                   # Entfernt eigenständige Zahlen
    message = re.sub('\w*\d\w*', '', message)                               # Entfernt Wörter, die Zahlen enthalten
    message = re.sub(' +', ' ', message)                                    # Reduziert mehrere Leerzeichen auf ein einzelnes Leerzeichen
    return message


In [ ]:
df['Content'] = df['Content'].apply(lambda x: clean_text(x))
df['Content'].head()

#### <span style="color: orange;">Entfernen von Begrüßungen und Schlussformen</span>

<p style="text-align: justify;"> 
In der nächsten Funktion werden bestimmte Wörter hinzugefügt, die in der Analyse aufgefallen sind und wenig zur Interpretation der Nachricht beitragen. Dazu gehören Begrüßungen, Endungen sowie Abschnitte mit verstecktem Code ("hidden code"), die manuell in der Funktion definiert wurden.
</p>

In [ ]:
def remove_greetings_and_closings(text):
    # Liste der Begrüßungen und Verabschiedungen die entfernt werden sollen unabhänig 
    # von der Position im  Text und besondere Kombis wie hiddenfalse/wlsdexception etc.
    phrases_to_remove = [
        "Dear", "Hi", "Hello", "Good morning", "Good evening",
        "Best regards", "best regard", "Kind regard", "Sincerely", "Yours truly",
        "All the best", "Thank you", "Mr.", "Ms.", "Mrs.", "Mr", "Ms", "Mrs", "bye", "mr", "ms", "mrs",
        "regard", "regards", "kindly", "kindly regards", "sincerely yours","hennadij", "nt", "verhoeven", "bye", "nbsp", "namemedium", "grid"
        "wlsdexception", "semihiddenfalse", "lockedfalse", "semihidden", "unhidewhenusedfalse", "unhidewhenusedtrue", 'nt', 'am','ag', "jv", "metallurg", "metallurgy"
    ]
    
    # Regex-Muster, um alle Begriffe zu entfernen
    pattern = r'\b(?:' + '|'.join(re.escape(phrase) for phrase in phrases_to_remove) + r')\b'
    
    # Begriffe aus dem Text entfernen
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    return text

In [ ]:
# Wende die Funktion auf die Spalte 'Content' an
df['Content'] = df['Content'].apply(remove_greetings_and_closings)
df['Content'].head()

Manueller Eingriff hat geklappt. Output bestätigt, dass das Entfernen dieser custom phrases_to_remove funktioniert hat.

####  <span style="color: orange;">Entfernen von Named Entities </span>

<p style="text-align: justify;"> 
Hauptäschlich ist es wichtig Namen von Personen zu entfernen, automatisch erkannte von Spacy aber auch vordefinierte die Spacy z.B. nicht erkennen kann. Zusätzlich Organisationen und Cardinal Zahlen welche den Wörtern noch mehr Gewicht geben und somit to Overfitting führen würden in der Modellierung.
</p>

In [ ]:
custom_names = ["Husar", "Koller", "Alex", "Chris", "Kramer", "Meyer", "mihalits"]

# Entferne benannte Entitäten (z. B. PERSON, ORG, CARDINAL)
def remove_named_entities_and_custom_names(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.ent_type_ not in ['PERSON', 'ORG', 'CARDINAL']]
    text_without_entities = " ".join(tokens)
    
    # Entferne benutzerdefinierte Namen
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in custom_names) + r')\b'
    text_without_custom_names = re.sub(pattern, '', text_without_entities, flags=re.IGNORECASE)
    
    return text_without_custom_names

In [ ]:
# Wende die Verarbeitung an
df['Content'] = df['Content'].apply(remove_named_entities_and_custom_names)
df['Content'].head()

Nach erfolgreichem Reinigen und Normalisieren des Datasets kann nur eine Kopie gespeichert werden welche für die Modellierung verwendet werden kann.

In [ ]:
# Speichern des "bereinigten" DataFrames in eine Excel-Datei
df_normalized = df.copy()
df_normalized.to_excel('df_normalized.xlsx', index=False)

<p style="text-align: justify;"> 
Nach der Bereinigung und Entfernung bestimmter Wörter, Sequenzen und Kombinationen verändert sich entsprechend die Wortanzahl einer Nachricht. Zu Beginn der Analyse wurde die durchschnittliche Satzlänge betrachtet, die bei 5 Wörtern lag. Nun untersuchen wir, wie viele Nachrichten weniger als 10 Wörter enthalten, da wir ab dieser Grenze davon ausgehen, dass eine Verhandlungsnachricht noch ausreichend Inhalt besitzt, um eine sinnvolle Mitteilung zu vermitteln.</p>

In [ ]:
# Leere Nachrichten printen da nach dem Bereinigen es sein kann dass einige Nachrichten leer sind
df[df['Content'] == '']

In [ ]:
#Entfernen der leeren Zeilen
df = df[df['Content'] != '']
df['Content'].isnull().sum() # checken ob noch leere Zeilen vorhanden sind

In [ ]:
for index, row in df.iterrows():
    print(f"Row {index}: {row['Content']}")

Es finden sich viele einzelne Buchstaben, Rechtschreibfehler sowie falsche Worttrennungen und fehlende Leerzeichen in den Nachrichten. Diese Probleme könnten im späteren Verlauf durch Lemmatization und Tokenisierung des Textes behoben werden. 

####  <span style="color: orange;">Autokorrektur anwenden</span> 

Bevor die eigentliche Verarbeitung des Contents stattfindet, ist es vorteilhaft, den Text auf Rechtschreibfehler zu korrigieren. In den Observationen können Fehler auftreten, die die Verarbeitung beeinträchtigen könnten.

In [ ]:
# verwenden der autocorrect package dauert seeeehr lange vorerst auskommentieren
#!pip install autocorrect

In [ ]:
#nimmt eine weile bis es den ganzen Text durchgegangen ist
from autocorrect import Speller

spell = Speller(lang='en')

df['Content'] = df['Content'].apply(lambda x: " ".join([spell(word) for word in x.split()]))
df['Content']

Nochmal sauber Machen

In [ ]:
df['Content'] = df['Content'].apply(lambda x: clean_text(x))
df['Content'].head()

### **<span style="color: orange;"> Tokenisierung  </span>**


<p style="text-align: justify;"> 
Tokenisierung bezeichnet die systematische Zerlegung eines Textes in kleinere textuelle Ein-heiten, die sogenannten Tokens. Dieser Prozess kann auf verschiedenen Ebenen stattfinden, beispielsweise auf Wort- oder Satzebene. Die Unterteilung erfolgt in der Regel anhand von bestimmten Regeln basierend auf Leerzeichen, Punkte oder Kommata (vgl. Datta et al. 2021, S. 3010; Kadhim et al. 2014, S. 70). Die nach der Tokenisierung entstandenen Wörter werden folglich als-Vektoren zur Datenmodellierung genutzt. Somit werden innerhalb der Daten ef-fektive Korrelationen hergestellt, was zu einer verbesserten Performanz Modellperformanz beiträgt (Datta et al., 2021, p. 3010).</p>

In [ ]:
# Importieren von SpaCy
import spacy

# Laden des SpaCy-Modells
nlp = spacy.load("en_core_web_sm")

# Tokenisierung mit SpaCy
def spacy_tokenize(text):
    if isinstance(text, str):  # Überprüfen, ob der Text ein String ist
        doc = nlp(text)  # Text mit SpaCy verarbeiten
        tokens = [token.text.lower() for token in doc if not token.is_punct]  # Token extrahieren und in Kleinbuchstaben umwandeln
        return tokens
    return text  # Falls kein String, den ursprünglichen Wert zurückgeben


In [ ]:
# Neue Spalte 'Tokenized_Content' erstellen mit Spacy 
df['Tokenized_Content'] = df['Content'].apply(spacy_tokenize)

# Ausgabe der ersten Zeilen zur Überprüfung
df[['Content', 'Tokenized_Content']].head()

###  **<span style="color: orange;"> Entfernung Stoppwörter  </span>**

<p style="text-align: justify;"> 
Stoppwörter sind häufig vorkommende Wörter mit geringem semantischem Wert, die in Text-dokumenten oft auftreten. Dazu gehören unter anderem grammatikalische Artikel wie „der“, „die“, „das“ sowie Konjunktionen wie „und“ oder „oder“. Da solche Wörter wenig zur inhalt-lichen Analyse beitragen, können sie den Kategorisierungsprozess stören und die Effizienz von Klassifikationsalgorithmen verringern (vgl. Al-Hawari and Barham 2021a, S. 4865) . Das Entfernen dieser Wörter führt oft zu besseren statistischen Ergebnissen, jedoch können dabei auch stilistische Merkmale der Sprache verloren gehen (vgl. Kadhim et al. 2014, S. 70).</p>

In [ ]:
#%import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

# Liste der englischen Stopwörter
stopwords_en = stopwords.words('english')
# zusätzliche Wörter und Buchstaben hinzufügen
additional_stopwords = ['nt', 'am','ag', "jv", "u"]
stopwords_en.extend(additional_stopwords)

In [ ]:
# Ausgabe der aktualisierten Stopwortliste
print(stopwords_en)

In [ ]:
# Entferne Stopwörter aus der Liste von Wörtern
def remove_stopwords(words):
    filtered_words = [w for w in words if w.lower() not in stopwords_en]
    return filtered_words 

In [ ]:
# Funktion anwenden
df['Content'] = df['Tokenized_Content'].apply(remove_stopwords)

In [ ]:
# Zeige die ersten Zeilen der bereinigten Daten
df[['Content', 'Tokenized_Content']].head()

### **<span style="color: orange;"> Stemming/ Stammformreduktion  </span>**
<p style="text-align: justify;"> 
Stammformreduktion, auch bekannt als „Stemming, ist eine Methode zur Reduzierung von Wörtern auf ihren Wortstamm, indem Suffixe abgeschnitten werden. Dabei wird weder der Kontext noch die Wortart berücksichtigt, was zu ungenauen oder sinnlosen Wortformen füh-ren kann. Ein Beispiel ist das Wort „studying“, das korrekt zu „study“ reduziert wird, wäh-rend „coding“ fälschlicherweise zu „cod“ gekürzt wird (vgl. Al-Hawari and Barham 2021a, S. 4865)</p>


In [ ]:
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

In [ ]:
# porter stemmer definieren 
ps = PorterStemmer()

In [ ]:
# Stemmen der Wörter und Speichern in einer neuen Spalte um aktuellen Content nicht zu verändern
df['Stemmed_Content'] = df['Content'].apply(lambda tokens: [ps.stem(word) for word in tokens])
df[['Content', 'Tokenized_Content', 'Stemmed_Content']].head()

Stemming ist eine schnellere Methode jedoch ist der Kontext für uns wichtig, daher verwenden wir die Lemmatization.

## <span style="color: orange;"> Überprüfung nach dem Cleansing um für Lemmatizing vorzubereiten</span>

In [ ]:
# Die Token-Liste in der 'Content'-Spalte wieder zu einem String zusammenfügen
df['Content'] = df['Content'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)

# Überprüfen, ob die Spalte korrekt zusammengefügt wurde
df['Content'].head()

In [ ]:
# Alle Wörter aus der Spalte 'Content' extrahieren und filtern, die aus einem Buchstaben bestehen
single_letter_words = df['Content'].str.split().explode().loc[lambda x: x.str.len() == 1]

In [ ]:
# Entfernen alle Wörter, die aus genau einem Buchstaben bestehen
df['Content'] = df['Content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

# Zählen der einzelnen Buchstaben
single_letter_counts = single_letter_words.value_counts()

# Überprüfen
print("\nHäufigkeit der einzelnen Buchstaben vor der Entfernung:")
print(single_letter_counts)

In [ ]:
# Entfernen der einzelnen Buchstaben
df['Content'] = df['Content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

In [ ]:
# Überprüfung ob noch einzelne Buchstaben vorhanden sind
single_letter_words_after = df['Content'].str.split().explode().loc[lambda x: x.str.len() == 1]
single_letter_counts_after = single_letter_words_after.value_counts()

print("\nHäufigkeit der einzelnen Buchstaben nach der Entfernung:")
print(single_letter_counts_after)

Nach der Vorverarbeitung und Reinigung des DataFrames sollte er erneut auf NaN-Werte und andere Auffälligkeiten überprüft werden, um ihn weiter zu normalisieren.
Zählen aller Content-Zeilen, die leer sind, anstatt sie als NaN zu behandeln, da im aktuellen Dataset leere Strings vorhanden sind, die nicht als NaN erkannt werden.

In [ ]:
# "leere" Zeilen zählen
df[df['Content'] == ''].shape[0] 

In [ ]:
# entfernen der leeren Zeilen
df = df[df['Content'] != '']

In [ ]:
# überprüfen 
df[df['Content'] == ''].shape[0] # 0 rows are empty

In [ ]:
df['NegoOutcome'].value_counts()

<p style="text-align: justify;"> 
Von den ursprünglich 7.026 Observationen wurden 11 entfernt, da sie keinen signifikanten Wert lieferten. Diese Observationen bestanden beispielsweise nur aus einem Wort oder enthielten leere Zeichen und wurden daher ausgeschlossen. 
</p>

###  **<span style="color: orange;"> Lemmatisierung  </span>**

<p style="text-align: justify;"> 
Lemmatisierung, bekannt als „Lemmatization“ ist ein Verfahren zur Reduzierung von Wörtern auf ihre Grundform, das sogenannte „Lemma“. Im Gegensatz zum Stemming wird hierbei der grammatikalische Kontext einbezogen, um die korrekte Grundform des Wortes zu bestimmen. 
Beispielsweise wird das Wort „leaves“ je nach POS-Kontext entweder auf „leave“ oder „leaf“ reduziert, während bei „better“ die Grundform „good“ erkannt wird (vgl. Cha 2023, S. 527). Diese verbesserte Genauigkeit macht die Lemmatisierung besonders nützlich, weil die Berück-sichtigung semantischer Bedeutung bei textuellen Daten ausschlaggebend für die Analyse ist (vgl. Chai 2023, S. 527f.; Khan et al. 2025, S. 19). </p>


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
# WordNetLemmatizer initialisieren
lemmatizer = WordNetLemmatizer()

In [ ]:
# Wieder neu tokenisieren um dann Lemmatisierung darauf anwenden zu können 
df['Tokenized_Content_new'] = df['Content'].apply(spacy_tokenize)
df[['Content', 'Tokenized_Content_new']].head()

In [ ]:
# Lemmatisieren der Wörter und Speichern in einer neuen Spalte um aktuellen Content nicht zu verändern
df['Lemmatized_Content'] = df['Tokenized_Content_new'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])
df[['Content', 'Tokenized_Content_new', 'Stemmed_Content','Lemmatized_Content']].head()

Nach dem genaueren Vergleich der unterschiedlcihen Tokenisierung und Wortverkürzungen wird mit Lemmatisierung weiter gemacht weil die Wörter hier mehr sinn ergeben als die Stammformreduktion.

In [ ]:
# lemmatisieren zusammenführen und mit Content ersetzen für alle Content Splits
df['Content'] = df['Lemmatized_Content'].apply(lambda tokens: ' '.join(tokens))
df[['Content', 'Lemmatized_Content']].head()

###  **<span style="color: orange;"> Valide Daten erforschen  </span>**

Im nächsten Schritt wird der bereinigte Datensatz gespeichert und in einem separaten Notebook, "Ex-Post Analyse Master", einem Ex-Post-Vergleich unterzogen. Dabei wird der ursprüngliche Datensatz ("initial") dem bereinigten ("cleansed") gegenübergestellt und grafisch analysiert, um die Veränderungen hervorzuheben. Dieser Vorher-Nachher-Vergleich bildet die Grundlage für die anschließende Transformation und Modellierung.

In [ ]:
# Überprüfen ob mind. 4 Wörter in Content vorhanden sind damit ein "Satz" daruas gebildet werden kann wenn danach in 75/50 und 25% gepslittet wird
# Überprüfen, ob mindestens 4 Wörter in der Spalte 'Content' vorhanden sind
df['Word_Count_after_cleansing'] = df['Content'].apply(lambda x: len(x.split()))  # Zählt die Wörter in jedem Eintrag
df_valid = df[df['Word_Count_after_cleansing'] < 4]  # Filtert nur die Einträge mit mindestens 4 Wörtern

# 4 Wörter kein semantischer Inhalt für Endaussage?
print(f"Anzahl der ungültigen Einträge (weniger als 4 Wörter): {len(df_valid)}")
print(f"Anzahl der gültigen Einträge: {len(df) - len(df_valid)}")

# Optional: Entfernen der ungültigen Einträge aus dem DataFrame
#df = df_valid



In [ ]:
# Printen der rows der ungültigen Einträge
for index, row in df_valid.iterrows():
    print(f"Row {index}: {row['Content']}")

In [ ]:
df_valid.head()

<p style="text-align: justify;"> 
In der obigen Analyse wurde festgestellt, dass es 69 Nachrichten gibt, die weniger als 4 Wörter enthalten. Das bedeutet, dass diese Nachrichten bei der Transformation und Analyse der gekürzten Texte automatisch in der Spalte Content als NaN-Werte angezeigt werden. Diese NaN-Werte werden entfernt, um sicherzustellen, dass der Prozess reibungslos abläuft. Die Grenze von 4 Wörtern wurde gewählt, da bei einer Reduktion auf 75 %, 50 % und 25 % des Textes die Anzahl der Wörter entsprechend abnimmt. Beispielsweise würde ein Text mit 4 Wörtern bei einer Reduktion auf 50 % nur noch 2 Wörter enthalten, und bei 25 % sogar nur 1 Wort. Solche kurzen Texte bieten keinen ausreichenden semantischen Inhalt für eine sinnvolle Analyse oder Modellierung.

In [ ]:
# Erstellen einer neuen DF für Observations die midnestens 4 Wörter enthalten
df_valid_content = df.copy()
df_valid_content = df_valid_content[df_valid_content['Word_Count_after_cleansing'] >= 4] # Filtert nur die Einträge mit mindestens 4 Wörtern
df_valid_content = df_valid_content.reset_index(drop=True) # Reset der Indizes
df_valid_content.head() # Ausgabe der ersten 5 Zeilen des neuen DataFrames
df_valid_content['Word_Count_after_cleansing'].describe() # Statistiken für die Wortanzahl

In [ ]:
df_valid_content.head()

In [ ]:
# Zählen der Zeilen mit weniger als 4 Wörtern
rows_with_less_than_4_words = df_valid_content[df_valid_content['Word_Count_after_cleansing'] < 4]
print(f"Anzahl der Zeilen mit weniger als 4 Wörtern: {len(rows_with_less_than_4_words)}")

In [ ]:
# Speichern in neue Excel 
df_valid_content.to_excel('df_valid_content.xlsx', index=False) # Speichern in Excel

Dieses Dataset kann nun für die Modellierung verwendet werden, da alle Content-Spalten die gleiche Anzahl an Observationen enthalten, ähnlich wie bei den 25%, wodurch eine konsistente Grundlage geschaffen wird.

### <span style="color: orange;">Partielles Training-Test Set Sample nach chronologischer Reinhenfolge</span> 

<p style="text-align: justify;"> 
In diesem Ansatz wird der Originaltext quantilbasiert in gekürzte Versionen (25 %, 50 %, 75 %) unterteilt. Grund hierfür ist die spätere Untersuchung im Kapitel 4. Daten-Transformation und 5. Daten-Modellierung, wie sich eine Reduktion des Textumfangs auf die Robustheit von Verhandlungsanalysen und Modellvorhersagen auswirkt. 

Durch die chronologische Kürzung der Texte lässt sich zusätzlich analysieren, ob frühe, mittlere oder späte Textabschnitte besonders entscheidend für den Verhandlungserfolg sind. Die Ergebnisse können Aufschluss darüber geben, ob verkürzte Texte für eine erfolgreiche Verhandlungsprognose und das Training von RNN-Modellen ausreichen oder ob vollständige Kontextinformationen notwendig sind. 
</p>



In [ ]:
# Funktion um Text zu kürzen: Fokus dass bei Wörter abgeschnitten wird damit es semantisch sinnvoll erhalten bleibt
def truncate_text(text, percent):
    words = text.split()
    cutoff = int(len(words) * percent)
    return ' '.join(words[:cutoff])

In [ ]:
# Cutten in 25%, 50%, 75% Quantile
df['Content_75%'] = df['Content'].apply(lambda x: truncate_text(x, 0.75))
df['Content_50%'] = df['Content'].apply(lambda x: truncate_text(x, 0.50))
df['Content_25%'] = df['Content'].apply(lambda x: truncate_text(x, 0.25))

In [ ]:
#Tokenisieren der gekürzten Texte auf das schon lemmatisierte Content
df['Lemmatized_Content_75%'] = df['Content_75%'].apply(word_tokenize)
df['Lemmatized_Content_50%'] = df['Content_50%'].apply(word_tokenize)
df['Lemmatized_Content_25%'] = df['Content_25%'].apply(word_tokenize)

In [ ]:
# Zusammenführen der gekürzten Texte in ursprüngliche Form und nicht als Tokens
df['Content_75%'] = df['Lemmatized_Content_75%'].apply(lambda tokens: ' '.join(tokens))
df['Content_50%'] = df['Lemmatized_Content_50%'].apply(lambda tokens: ' '.join(tokens))
df['Content_25%'] = df['Lemmatized_Content_25%'].apply(lambda tokens: ' '.join(tokens))

In [ ]:
# Printen der ersten 2 Zeilen um zu zeigen dass es funktioniert hat
print("Truncated Texts:")
for index, row in df.head(2).iterrows():
    print(f"\n--- Row {index} ---")
    print(f"Original (100%): {row['Content']}")
    print(f"Truncated (75%): {row['Content_75%']}")
    print(f"Truncated (50%): {row['Content_50%']}")
    print(f"Truncated (25%): {row['Content_25%']}")


<p style="text-align: justify;">
Wie die Ergebnisse zeigen, wurde die Datenpartitionierung erfolgreich umgesetzt. Dabei wurde der Text jeder Verhandlung nach chronologischer Reihenfolge in Quantil-basierte Abschnitte (25%, 50%, 75%) unterteilt. Folglich wurde kein Wort abgeschnitten, sodass die Bedeutung aller Aussagen erhalten bleibt. 

In [ ]:
# Plotten der Verteilung der Wortanzahl bei unterschiedlichen Kürzungsgraden
df['Length_100'] = df['Content'].apply(lambda x: len(x.split()))
df['Length_75'] = df['Content_75%'].apply(lambda x: len(x.split()))
df['Length_50'] = df['Content_50%'].apply(lambda x: len(x.split()))
df['Length_25'] = df['Content_25%'].apply(lambda x: len(x.split()))

plt.figure(figsize=(10, 6))
plt.hist(df['Length_100'], bins=30, alpha=0.4, label='100%')
plt.hist(df['Length_75'], bins=30, alpha=0.4, label='75%')
plt.hist(df['Length_50'], bins=30, alpha=0.4, label='50%')
plt.hist(df['Length_25'], bins=30, alpha=0.4, label='25%')
plt.legend()
plt.title('Verteilung der Wortanzahl bei unterschiedlichen Kürzungsgraden')
plt.xlabel('Anzahl Wörter')
plt.ylabel('Häufigkeit')
plt.show()


<p style="text-align: justify;">
Die Grafik veranschaulicht deutlich, wie die Wortanzahl mit zunehmendem Kürzungsgrad (75% → 50% → 25%) abnimmt. Besonders interessant wird die Analyse der Auswirkungen dieser Reduktion auf die Modellperformance – beispielsweise ob bereits 25% des Textes ausreichen, um vergleichbare Ergebnisse zu erzielen.

In [ ]:
df.head()

In [ ]:
#Welche row enthält leere strings
# welche row hat nan werte
df[df['Content_75%'] == ''] # leere Strings
# df[df['Content_50%'] == ''] # leere Strings
# df[df['Content_25%'] == ''] # leere Strings

<p style="text-align: justify;">
Das bestätigt, dass leere Strings entstehen, weil der Originaltext in Quantile (75%, 50%, 25%) unterteilt wird. Daher werden bei der Transformation und Modellierung der Daten für jedes Quantil alle Zeilen (Rows) entfernt, die leer sind. Dies gewährleistet, dass auch die geringere Textmenge in den kürzeren Quantilen noch berücksichtigt wird und die Analyse konsistent bleibt.

In [ ]:
# droppen der rows wie bei valid die weniger als 4 Wörter haben
df_valid_new = df.copy()
df_valid_new = df_valid_new[df_valid_new['Word_Count_after_cleansing'] >= 4] # Filtert nur die Einträge mit mindestens 4 Wörtern
df_valid_new = df_valid_new.reset_index(drop=True) # Reset der Indizes
df_valid_new.head() # Ausgabe der ersten 5 Zeilen des neuen DataFrames
df_valid_new['Word_Count_after_cleansing'].describe() # Statistiken für die Wortanzahl

In [ ]:
# Zählen der Zeilen mit weniger als 4 Wörtern
rows_with_less_than_4_words = df_valid_new[df_valid_new['Word_Count_after_cleansing'] < 4]
print(f"Anzahl der Zeilen mit weniger als 4 Wörtern: {len(rows_with_less_than_4_words)}")

In [ ]:
# Speichern in neue Excel 
df_valid_new.to_excel('df_valid_new.xlsx', index=False) # Speichern in Excel

<p style="text-align: justify;">
Nun ist das vollständig überarbeitete Dataset bereinigt, und die notwendigen Komponenten wurden erstellt, um mit der Transformation sowie der Modellierung fortzufahren. Es ist ersichtlich, dass der Inhalt in seine Quantile unterteilt wurde, wobei jedes Quantil die chronologische Reihenfolge von der ersten Nachricht bis hin zu dem jeweiligen Quantil widerspiegelt.



In [ ]:
# Kopie erstellen für Ex-Post Vergleich und spätere Modellierung
df_complete_cleansing = df.copy()
df_complete_cleansing.to_excel('df_complete_cleansing.xlsx', index=False)

# Folgende Säulen behalten:
columns_to_keep = ['SenderID', 'ReceiverID', 'ActionType', 'NegoOutcome', 'Content', 'NegoOutcomeLabel', 'Lemmatized_Content', 'Content_25%', 'Content_50%', 'Content_75%', 'Lemmatized_Content_75%' , 'Lemmatized_Content_50%', 'Lemmatized_Content_25%']

# Neues DataFrame mit den custom-Spalten 
df_complete_cleansing_columns = df_complete_cleansing[columns_to_keep]
df_complete_cleansing_columns.to_excel('df_complete_cleansing_columns.xlsx', index=False)

In [ ]:
# Zeitstempel fürs Ende
end_time = time.time()

In [ ]:
# Ausführungszeit berechnen und ausgeben
execution_time = end_time - start_time

execution_time_minutes = execution_time // 60
execution_time_seconds = execution_time % 60

print(f"Die Ausführungszeit des Skripts beträgt: {int(execution_time_minutes)} Minuten und {execution_time_seconds:.2f} Sekunden")

# References


- Al-Hawari, F., Barham, H., 2021a. A machine learning based help desk system for IT service management. J. King Saud Univ. - Comput. Inf. Sci. 33, 702–718. https://doi.org/10.1016/j.jksuci.2019.04.001
- Al-Hawari, F., Barham, H., 2021b. A machine learning based help desk system for IT service management. J. King Saud Univ. - Comput. Inf. Sci. 33, 702–718. https://doi.org/10.1016/j.jksuci.2019.04.001
- Chai, C.P., 2023. Comparison of text preprocessing methods. Nat. Lang. Eng. 29, 509–553. https://doi.org/10.1017/S1351324922000213
- Datta, A., Jena, B., Dash, A.K., Pradhan, R., 2021. A Comprehensive Analytical Study of Traditional and Recent Development in Natural Language Processing. Int. J. Adv. Trends Comput. Sci. Eng. 10, 3009–3019. https://doi.org/10.30534/ijatcse/2021/121052021
- De Mulder, W., Bethard, S., Moens, M.-F., 2015. A survey on the application of recurrent neural networks to statistical language modeling. Comput. Speech Lang. 30, 61–98. https://doi.org/10.1016/j.csl.2014.09.005
- Kadhim, A.I., Cheah, Y.-N., Ahamed, N.H., 2014. Text Document Preprocessing and Dimen-sion Reduction Techniques for Text Document Clustering, in: 2014 4th International Conference on Artificial Intelligence with Applications in Engineering and Technolo-gy. Presented at the 2014 Artificial Intelligence with Applications in Engineering and Technology (ICAIET), IEEE, Kota Kinabalu, Malaysia, pp. 69–73. https://doi.org/10.1109/ICAIET.2014.21
- Karanikolas, N., Manga, E., Samaridi, N., Tousidou, E., Vassilakopoulos, M., 2023. Large Language Models versus Natural Language Understanding and Generation, in: Pro-ceedings of the 27th Pan-Hellenic Conference on Progress in Computing and Infor-matics. Presented at the PCI 2023: 27th Pan-Hellenic Conference on Progress in Com-puting and Informatics, ACM, Lamia Greece, pp. 278–290. https://doi.org/10.1145/3635059.3635104
- Kaya, M.-F., 2022. Pattern Labelling of Business Communication Data. Group Decis. Negot. 31, 1203–1234. https://doi.org/10.1007/s10726-022-09800-2
- Khan, J., Ahmad, K., Jagatheesaperumal, S.K., Sohn, K.-A., 2025. Textual variations in social media text processing applications: challenges, solutions, and trends. Artif. Intell. Rev. 58, 89. https://doi.org/10.1007/s10462-024-11071-z
- Khurana, D., Koli, A., Khatter, K., Singh, S., 2023. Natural language processing: state of the art, current trends and challenges. Multimed. Tools Appl. 82, 3713–3744. https://doi.org/10.1007/s11042-022-13428-4
- Zhang, Y., 2024. A New Method for Improving the Accuracy of Word Segmentation in Modern Chinese Texts, in: Chen, C. (Ed.), Frontiers in Artificial Intelligence and Applications. IOS Press. https://doi.org/10.3233/FAIA231409
